In [1]:
print('starting!')
import time
total_start = time.time()
import sys
import json
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
print('after transformers imports, time:', time.time() - total_start)
import numpy as np
import torch
import pandas as pd
from tqdm import tqdm
import unicodedata
import pickle
import concurrent.futures
import csv

USE_ONNX = False

if USE_ONNX:
    sys.path.append('/kaggle/input/python-packages/python_packages')
    import onnxruntime

print('after all imports (inc. torch), time:', time.time() - total_start)

NAMES_SELECTION = 1 #1, 2 or None - change this to use different names lists or not to use names list at all

# data_filepath = 'data/test.json'
# data_filepath = 'data/train.json'
data_filepath = '/kaggle/input/pii-detection-removal-from-educational-data/test.json'
# data_filepath = '/kaggle/input/pii-detection-removal-from-educational-data/train.json'

base_model = 'microsoft/mdeberta-v3-base'
inference_model_name = '/kaggle/input/xs2-ft3ds-3e-l64-2889/xs2_ft3ds_on_3dscheckpoint-29630_len64_3e_gelu_checkpoint-2889'
inference_model_name2 = '/kaggle/input/xs-ft3ds-2e-l64-1926/xs_ft3ds_on_903ds_2e_0.928_len64_gelu_checkpoint-1926'

### other models
# inference_model_name2 = '/kaggle/input/92-2-ft-on-3ds-xsmall/92_ft_on_3ds_xsmall_checkpoint-3059' # older, better performing model
# inference_model_name2 = '/kaggle/input/nimp2_cni1/pytorch/nimp3/1/checkpoint-34884'               # single-phase training
# inference_model_name2 = '/kaggle/input/nimp2_cni1/pytorch/nimo6_c2/1/checkpoint-1926'             # strong and easy to train
# inference_model_name2 = '/kaggle/input/nimp2_cni1/pytorch/conly_e3/1/checkpoint-2889'             # only competition data
# inference_model_name = '/kaggle/input/nimp2_cni1/pytorch/conly_xs2_e3/1/checkpoint-2889'          # only competition data smaller model
# inference_model_name = '/kaggle/input/nimp2_cni1/pytorch/xs2_nimo5_c2/1/checkpoint-1926'          # strong and easy to train smaller model
###
base_model = inference_model_name

onnx_model20 = '/kaggle/input/onnx-l64-q-o3/xs_len64_onnx_q_o3/model_quantized_optimized.onnx' # based on: xs_ft3ds_on_903ds_2e_0.928_len64_gelu_checkpoint-1926
onnx_model = '/kaggle/input/onnx-xs2-l64/xs2_len64_onnx/model.onnx'                            # based on: xs2_ft3ds_on_3dscheckpoint-29630_len64_3e_gelu_checkpoint-2889


df = pd.read_json(data_filepath)

train_dataset = Dataset.from_pandas(df, preserve_index=False)
dataset = DatasetDict({'train': train_dataset})
print(dataset)
print(len(dataset['train']))

if len(dataset['train']) < 1000:
    data_filepath = '/kaggle/input/pii-detection-removal-from-educational-data/train.json'
    print('It is not a submission! Overriding test dataset to train.json!')
    df = pd.read_json(data_filepath)
    train_dataset = Dataset.from_pandas(df, preserve_index=False)
    dataset = DatasetDict({'train': train_dataset})    
    print(dataset)
    print(len(dataset['train']))

original_dataset = dataset
original_df = df

document_to_rows = {}
for i, row in tqdm(original_df.iterrows()):
    document_to_rows[row['document']] = row


#---------------------------------------------------------------------- add id_num directly to csv based on simple rule
start = time.time()
csv_simple_rule_rows = []

# print('normalizing...')
# #df['tokens'] = df['tokens'].str.normalize('NFKD') 
# df['tokens'] = df['tokens'].apply(lambda tokens: [unicodedata.normalize('NFKD', token) for token in tokens])
# print('normalized')

for index, row in df.iterrows():
    names = []
    namesi = []
    for i,token in enumerate(row.tokens):
        if len(token[-6:]) >= 6 and token[-6:].isnumeric() and 'http' not in token:
            names.append(token)
            namesi.append(i)
    
    for i, token in zip(namesi, names):
        if 'x' in token and token.replace('x','').isdigit() or ')' in token or '.' in token and token.replace('.','').isdigit():
            print('PHONE_NUM: ', i, token)
            fake_label = 'I-PHONE_NUM'
        elif len(token) == 5 and token.isdigit():
            print('ADDRESS: ', i, token)
            fake_label = 'I-STREET_ADDRESS'
        else:
            fake_label = 'B-ID_NUM'
            print('ID_NUM: ', i, token)
        csv_simple_rule_rows.append([0, row.document, i, fake_label])

print('csv_simple_rule_rows for ID_NUM populated: ', len(csv_simple_rule_rows))

# sys.exit()
stop = time.time()
print(csv_simple_rule_rows)
print('csv_simple_rule_rows for ID_NUM time: ', time.time() - start)
#----------------------------------------------------------------------



print('building all_lower_set...')
all_lower = []
for index, row in df.iterrows():
    lower_tokens = [token for token in row.tokens if token.islower()]
    all_lower.extend(lower_tokens)
all_lower_set = set(all_lower)



local_files_only_flag = False
tokenizer1 = AutoTokenizer.from_pretrained(base_model, local_files_only=local_files_only_flag, additional_special_tokens=['[SP]', '[NL]'])
tokenizer20 = AutoTokenizer.from_pretrained(inference_model_name2, local_files_only=local_files_only_flag, additional_special_tokens=['[SP]', '[NL]'])

tokenizer = tokenizer1


labels = ['B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL', 'O']
labels = labels[-1:] + labels[:-1] #for convenience, so O will be at index 0

id2label = {}
label2id = {}
for i, label in enumerate(labels):
    id2label[i] = label
    label2id[label] = i


tokenizer_kwargs = {'max_length': 64, 'padding': 'max_length', 'truncation': True, 'return_overflowing_tokens': True, 'stride':12 }

def tokenize_function(example):
    start = time.time()
    tokens_batch = example['tokens']
    tokens_batch = [[t.replace('\n', '[NL]').replace(' ', '[SP]').replace('\u200b', '') for t in tokens] for tokens in tokens_batch] #TODO it assumes batches only
    result = tokenizer(tokens_batch, **tokenizer_kwargs, return_tensors='np', is_split_into_words=True) #np vs pt
    stop = time.time()
    print('tokenize_function time: ', stop - start)
    return result

def tokenize_and_align_original_columns_new(examples):
    tokenized_inputs = tokenize_function(examples)

    word_ids_batch = []
    tokens = []
    full_text = []
    document = []
    
    input_ids = []
    token_type_ids = []
    attention_mask = []
    overflow_to_sample_mapping = []
    tokens_list = examples['tokens']
#     np.object = object #problem was in lack of np.asarray
    for i, original_i in enumerate(tokenized_inputs['overflow_to_sample_mapping']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
#         word_ids_batch.append(word_ids)
        
        word_ids_non_none = [id for id in word_ids if id is not None]
        first_word_id = word_ids_non_none[0]
        last_word_id = word_ids_non_none[-1]
        
        tt = tokens_list[original_i][first_word_id:last_word_id+1]
        if not check_for_pii(tt):
            continue
        
        word_ids_batch.append(word_ids)
        tokens.append(examples['tokens'][original_i][first_word_id:last_word_id+1]) #that first/last wasnt here before
        full_text.append(examples['full_text'][original_i])
        document.append(examples['document'][original_i])
        
        input_ids_i = tokenized_inputs.input_ids[i]
        input_ids.append(input_ids_i)
        token_type_ids_i = tokenized_inputs.token_type_ids[i]
        token_type_ids.append(token_type_ids_i)
        attention_mask_i = tokenized_inputs.attention_mask[i]
        attention_mask.append(attention_mask_i)
        overflow_to_sample_mapping_i = tokenized_inputs.overflow_to_sample_mapping[i]
        overflow_to_sample_mapping.append(overflow_to_sample_mapping_i)

    tokenized_inputs['word_ids'] = word_ids_batch
    tokenized_inputs['tokens'] = tokens
    tokenized_inputs['full_text'] = full_text
    tokenized_inputs['document'] = document
    
    tokenized_inputs['input_ids'] = np.asarray(input_ids)
    tokenized_inputs['token_type_ids'] = np.asarray(token_type_ids)
    tokenized_inputs['attention_mask'] = np.asarray(attention_mask)
    tokenized_inputs['overflow_to_sample_mapping'] = np.asarray(overflow_to_sample_mapping)
    
    return tokenized_inputs


example = dataset['train'][0]
tokenized_example = tokenize_function(example) #fire once so caching works

columns_to_remove = ['trailing_whitespace', 'full_text', 'document', 'tokens']
if 'labels' in dataset['train'][0]:
    columns_to_remove.append('labels')

print('columns_to_remove: ', columns_to_remove)

tokenizer = tokenizer1
dataset20 = dataset


print('dataset:')
print(dataset)
print('dataset20:')
print(dataset20)



start = time.time()

if NAMES_SELECTION == 1:
    all_names_file = '/kaggle/input/pii-real-names-dataset/given_names_data.parquet'
    names7 = pd.read_parquet(all_names_file, columns=['given_name'])
    names7 = names7['given_name']
    names7 = [name for name in names7 if name != None]
    names7 = set(names7)
    all_names = names7
    print('all_names contains: ', len(all_names), 'names')
elif NAMES_SELECTION == 2:
    #N7
    all_names_file = '/kaggle/input/all-namesn7c/all_namesN7C.pkl'
    with open(all_names_file, 'rb') as f: 
        namesN7C = pickle.load(f)
    all_names = namesN7C

    #N7
    all_names_file = '/kaggle/input/all-namesw/all_namesW.pkl'
    with open(all_names_file, 'rb') as f: 
        namesW = pickle.load(f)
    all_names |= namesW

    print('all_names contains: ', len(all_names), 'names')
else:
    all_names = None
    print('proceeding without names lists')

stop = time.time()
print('all_names load time: ', stop - start)

start = time.time()

stop_words1 = ['We', 'They', 'This', 'Our', 'Out', 'The', 'To', 'How', 'He', 'Then', 'By', 'Also', 'Here', 'Mind', 'Story', 'Tool', 'You', 'In', 'When', 'If', 'Not', 'Some', 'My', 
              'After', 'Since', 'Why', 'As', 'Did', 'One', 'So', 'There', 'Once', 'Before', 'But']
stop_words2 = ['A', 'At', 'All', 'From', 'Next', 'On', 'First', 'Innovation', 'Example', 'Most', 'India', 'Visual', 'Many', 'Thus', 'Based', 'Assignment', 'She', 'Hence', 'Even', 'Map', 'An', 'Every', 'Given', 'Is', 'Data', 
'Sales', 'Due', 'Being', 'Use', 'Journey', 'English', 'No', 'New', 'Future', 'Well', 'Value', 'Second', 'More', 'Do', 'Moving', 'According', 'Final', 'Human', 'Good', 'Firstly', 'Post', 'Like', 'Key', 'Other',
 'Step', 'Idea', 'Just', 'School', 'Time', 'Overall', 'Indian', 'Let', 'Chain', 'App', 'Finance', 'Both', 'Maps', 'Make', 'Later', 'Bank', 'Despite', 'Page', 'Agile', 'Can', 'Name', 'Going', 'Peer', 'Last', 
 'Brand', 'Ideas', 'Brazil', 'Over', 'Quality', 'Student', 'Tools', 'General', 'South', 'Director', 'Her', 'Life', 'Are', 'Me', 'Congress', 'Be', 'Africa', 'Rather', 'Or', 'Analysis', 'Knowledge', 'Senior',
 'Cost', 'Power', 'Along', 'Thank', 'Maybe', 'Growth', 'Head', 'State', 'Plan', 'Works', 'Lack', 'Define', 'Europe', 'Any', 'Low', 'Among', 'Stage', 'Regarding', 'Non', 'Work', 'Seeing', 'Phase', 'Mexico', 'Big', 'Have', 'Start', 'America', 'Kitchen', 'Apart',
 'Take', 'Day', 'African', 'Co', 'Part', 'Focus', 'Risk', 'Children', 'Does', 'Yet', 'Transformation', 'Right', 'Smart', 'Parents', 'Date', 'Level', 'Lead', 'Excel', 'Core', 'Media',
 'Case', 'Current', 'Drawing', 'Few', 'Care', 'Peru', 'Point', 'Cloud', 'Zoom', 'States', 'Teachers', 'Better', 'March', 'Note', 'End', 'Master', 'Delivery', 'Develop', 'Main', 'China', 'Web',
 'Very', 'Chinese',  'Prior', 'Apple', 'Real', 'Self', 'Platform', 'East', 'Small', 'Impact', 'September', 'Get', 'Elements', 'Spanish', 'Singapore', 'Vision', 'Canvas',
 'Wow', 'Ask', 'Back', 'October', 'Simple', 'Industrial', 'Australia', 'December', 'Add', 'Lean', 'January', 'Central', 'Go',
 'What', 'People', 'Company', 'Project', 'Internet', 'Video', 'Your', 'Two', 'Experience', 'Only', 'High',  'United', 'City', 'Third', 'Science', 'American', 'Together', 'Keeping', 'Professor', 'World', 'Three',
 'Mr',
#  'Apt',
 ]
 #'Mr',
    
stop_words3 = [ 'Pakistan',  'Red', 'Intelligence', 'Nigeria', 'Task', 'Keep', 'Increase', 'Staff', 'Had', 'Soon', 'June', 'Latino', 'Amazon', 
    'November', 'Peace', 'Imagine', 'Centre', 'North', 'Live', 'Home', 'French', 'Need', 'Persona', 'See', 'York', 'Put', 'Ans', 'Cash', 'Flow', 'Pre', 'Best', 'Partner', 'Price', 'Author', 
    'Was', 'Fast', 'Job', 'Still', 'Farmers', 'Content', 'Study', 'Argentina', 'Play', 'Lego', 'Find', 'Open', 'Field', 'Spain', 'Share', 'July', 'German', 'San', 'Little', 'Oil', 'Free', 
    'August', 'Great', 'Leaders', 'Target', 'Loan', 'Similar', 'Minimum', 'Should', 'Graphic', 'Russia', 'Germany', 'Same', 'Various', 'Coffee', 'Pilot', 'Talent', 'Give', 'Selected', 
    'Shop', 'Hiring', 'Club', 'Middle', 'Lab', 'Voice', 'Around', 'Culture', 'Act', 'Game', 'Plus', 'Friday', 'Up', 'Gas', 'Brands', 'Title', 'Jeanne', 'Top', 'Brain', 'Area', 'Major', 
    'Wi', 'Fi', 'Presentation', 'Air', 'Word', 'Six', 'Draw', 'Banks', 'France', 'Method', 'Physical', 'List', 'Korea', 'Image', 'Energy', 'Campus', 'Scientific', 'British', 'Beyond', 'Long', 
    'Show', 'None', 'Fig', 'Root', 'Net', 'Goal', 'Monday', 'Present', 'Steps', 'Try', 'Set', 'Faculty', 'Age', 'Select', 'Majority', 'Russian', 'Medium', 'Material', 'Asking', 
    'Moscow', 'Patients', 'Tesla', 'Straw', 'Much', 'Allow', 'Made', 'Greek', 'Success', 'February', 'Pin', 'Sector', 'Christmas', 'Tell', 'Water', 'Line', 'Selling', 'Traditional', 
    'Used', 'Person', 'Reason', 'Cross', 'Urban', 'Less', 'Rapid', 'Beauty', 'Choice', 'Excellence', 'Four', 'Discovery', 'Innovative', 'Table', 'Canada', 'Coaching', 'Choose',
    'Agents', 'Local', 'Too', 'Saturday', 'Lagos', 'Role', 'Mental', 'Kenya', 'Portal', 'Law', 'Lunch', 'Higher', 'Matter', 'Critical', 'Short', 'Videos', 'Ecuador', 'Accounting', 
    'Jobs', 'Suddenly', 'Car', 'Call', 'Re', 'Cyber', 'Pension', 'Vice', 'Chile', 'Avoid', 'Asset', 'Large', 'Force', 'Mural', 'Standard', 'Positive', 'Child', 'Card', 
    'Honestly', 'Consider', 'Blue', 'Rimini', 'Ghana', 'Zone', 'Read', 'Multi', 'Room', 'Avion', 'Bachelor', 'Firm', 'Trust', 'Fortune', 'Channel', 'God', 
    'Transport', 'Sale', 'Index', 'Purpose', 'Solar', 'Kids', 'Interview', 'Ref', 'Paris', 'Adobe', 'Score', 'Half', 'Western', 'Book', 'Offer', 'Changes', 'Primary', 'Eastern',
    'Coach', 'Ex', 'Speed', 'Doctors', 'Region', 'Grade', 'Capital', 'Stress', 'Black', 'Delhi', 'Front', 'Greece', 'Face', 'Tree', 'Radio', 'Hong', 'Kong', 'Serious', 'Topic', 
    'Stanford', 'Discover', 'Laundry', 'Early', 'Active', 'Sample', 'Till', 'London', 'Alpha', 'Pitch', 'Look', 'Carbon', 'Gender', 'Return', 'Uber', 'Probably', 'Member', 'Year', 'Divine', 
    'Manual', 'Teacher', 'Uruguay', 'Previous', 'Drive', 'Press', 'Bus', 'Kingdom', 'Switzerland', 'Airport', 'Answer', 'Clear', 'Patient', 'Balance', 'Vehicle', 'True', 'Engine', 'Sr', 'Maersk', 
    'Native', 'Hotel', 'Challange', 'Everyday', 'Poor', 'Situation', 'Fail', 'Buyer', 'Device', 'Principal', 'Nature', 'Modern', 'Desing', 'Natural', 'Done', 'Quito', 'Logo', 
    'Slowly', 'Win', 'Tuesday', 'Park', 'Einstein', 'Basic', 'Pay', 'Buy', 'Movie', 'Buddy', 'Hi', 'Lecture', 'Were', 'Quite', 'Past', 'Deep', 'Break', 'Color', 'Breaking', 'Anyway', 
    'Lots', 'Gain', 'Quick', 'Math', 'Dev', 'Disney', 'Masters', 'Town', 'Remote', 'Juniors', 'Bengali', 'Economy', 'Property', 'Record', 'Perfect', 'Normal', 'Coca', 'Cola', 
    'Parks', 'Cluster', 'Abu', 'Location', 'View', 'Provider', 'Mid', 'Unique', 'Than', 'Degree', 'Chemical', 'Anti', 'Happy', 'Anxiety', 'Traffic', 
    'Ways', 'Packaging', 'Possible', 'Follow', 'Release', 'Might', 'Commerce', 'Aristotle', 'Circle', 'Admin', 'Mumbai', 'Verso', 'Bitcoin', 'Maintain', 'Parts', 'Profit', 'Has', 
    'Journal', 'Guide', 'Off', 'La', 'Stream', 'Latinos', 'Tax', 'Agreement', 'Moderate', 'Full', 'Proof', 'Box', 'Kit', 'Learners', 'Northern', 'Mission', 'Gen', 'Uganda', 'Etc', 
    'Leader', 'Peruvian', 'Double', 'House', 'Indians', 'Bora', 'Chapter', 'Wednesday', 'Sri', 'Jira', 'Pressure', 'Iran', 'Drug', 'Micro', 'Eg', 'Zero', 'Meet', 'Pueblo', 'Demo', 
    'São', 'Pi', 'Loss', 'Practical', 'Decide', 'Motor', 'Stock', 'Rio', 'Renaissance', 'Link', 'Founder', 'Lake', 'Roll', 'Owner', 'Train', 'Arabia', 'Volume', 'Arab', 'Log', 
    'Strong', 'Madrid', 'Artists', 'Hub', 'Azerbaijan', 'Stay', 'Feb', 'Station', 'Say', 'Ten', 'Arts', 'Jordan', 'Brush', 'Potential', 'Integrated', 'Print', 'Yard', 'Aires', 'Rome', 
    'Either', 'Fall', 'Triple', 'Kia', 'Pain', 'Round', 'Mood', 'Hand', 'Stick', 'Base', 'Daily', 'Proper', 'Lesson', 'Pillar', 'Hat', 'Tea', 'Lanka', 'Fourth', 'Bit', 'Oral', 'Capture', 'Dutch', 
    'Got', 'Complete', 'Hire', 'Bring', 'Burundi', 'Civil', 'Tank', 'Never', 'Detail', 'Assurance', 'Court', 'Persian', 'Mens', 'Amsterdam', 'Battery', 'Running', 
    'Confidence', 'Instructor', 'Pacific', 'Areas', 'Trial', 'Advisory', 'Pick', 'Climate', 'Code', 'Progress', 'Garden', 'Men', 'Way', 'Political', 'Happiness', 'Sony', 'Millennial', 'Medicine',
    'Puerto', 'Listen', 'Alternate', 'Fixed', 'Buenos', 'Suite', 'Inspiring', 'Berlin', 'Shoe', 'Excellent', 'Size', 'Mall', 'Okay', 'Direction', 'Begin', 'Printing',
    'Oh', 'Regular', 'Sea', 'Summit', 'Ops', 'Loans', 'Sponsor', 'Vinci', 'Option', 'Beta', 'Living', 'Text', 'Forward', 'Sun', 'Frame', 'Gap', 'Namibia', 'Wall', 
    'Debit', 'Chaplin', 'Am', 'Skill', 'Hood', 'Collect', 'Taiwan', 'Item', 'Bay', 'Brussels', 'Barcelona', 'Bike', 'Love', 'Mirror', 'Tourism', 'Pride', 'Trainer', 
    'Days', 'Ishikawa', 'Vietnam', 'Bid', 'Texas', 'Makes', 'Potter', 'Reverse', 'Sketch', 'Golden', 'Nov', 'Baby', 'Feel', 'Rs', 'Min', 'Nigerian', 'Investors', 
    'Port', 'Ability', 'Hard', 'Surely', 'Manufacturer', 'Blind', 'Light', 'Display', 'Bar', 'Quickly', 'Knapp', 'Citizen', 'Chronic', 'Lets', 'Lines', 
    'Come', 'Bible', 'Pattern', 'Deliver', 'Bloom', 'Close', 'Successful', 'Trash', 'Theme', 'Yoga', 'Telco', 'Diamond', 'Seek', 'Dairy', 'Lawn', 'Window', 'Myanmar', 'Essential', 'Kind',
    'Continue', 'Reaction', 'Deutsche', 'Cart', 'War', 'Century', 'Maths', 'Regards', 'Ok', 'Enter', 'Bello', 'Ancient', 
    'Wallet', 'Cooking', 'Shift', 'Goods', 'Grow', 'Thursday', 'Java', 'Los', 'Times', 'Highly', 'Audit', 'Clarity', 'Mode', 'Ever', 'Toulouse', 'Sudan', 'Leave', 'Contracting', 
    'Munich', 'Late', 'Parking', 'Able', 'Save', 'Body', 'Constant', 'Welcome',  'Ford', 'Bi', 'Mobility', 'Scientist', 'Grand', 'Levels', 'Server', 'Easter', 'Cafe',
    'Guatemala', 'Ifs', 'Wish', 'Road', 'River', 'Linear', 'Bot', 'Pie', 'Empathise', 'Temp', 'Fun', 'Don', 'Faster', 'Quiz', 'Significant', 'Vary', 'Leisure', 
    'Via', 'Connect', 'Bengal', 'Flip', 'Smartphone', 'Flexible', 'Milan', 'Finca', 'Forest', 'Upcoming', 'Bundle', 'Believe', 'Valley', 'Guest', 'Kaiser', 'Millions',
    'Coke', 'Canal', 'Benz', 'Importantly', 'Enough', 'Ideal', 'Mol', 'Semi', 'Vs', 'Telekom', 'Speaker', 'Parent', 'Hero', 'Trip', 'Journeys', 'Mac', 
    'Rotaract', 'Prefect',  'Racing', 'Jefferson', 'Soldiers', 'Charity', 'Artist', 'Rooms', 'Huge', 'Dec', 'Nairobi', 'Africans', 'Respect', 'Janeiro', 'Wise', 'Mein', 
    'Olympic', 'Moral', 'Lecturer', 'Missing', 'Hot', 'Colour', 'Counselor', 'Doc', 'Sure', 'Them', 'Affordable', 'Mentor']
    #'Mars', 'Myer', 'Will', 'Meyer', 'Geoff', 'Miro', 'April', 'Ed', 'Andre', 'Tim', 'Alice', 'Hesse', 'Marian', 'Madeleine', 'Venn', 'Petty', 'Sarkar', 'Rob', 'Jane', 'Rya', 'Hellen', 'Norman', 
    #'Georgia', 'Hess', 'Paulo', 'Mark', 'Dan', 'Frank', 'Prezi', 'Ravi', 'Tom', 'Moses', 'Joana', 'Bob', 'Vicente', 'Eric', 'Victor', 'Jim', 'Louis', 'Tommy', 'Stefan', 'Rik', 'Bharti', 'Albert', 
    #'Kearney', 'Barry', 'Mercedes', 'Johari', 'Rose', 'Kerala', 'Sindhi', 'Davis', 'Lotte', 'Alexander', 'Henry', 'Dean', 'Hill', 'Mayer', 'Ramon', 'Tienda', 'Pago', 'Indus', 
    #'Charlie', 'Muller', 'Maori', 'Medellin', 'Marias', 'Sinek', 'Bono', 'Pragati', 'Mason', 'Edward', 'Luke', 'Churro', 'Malak', 'Wema', 'Hall', 'Sacco', 
    #'Anthony',
    #'Kelley', 'Keller', 'Rica', 'Pune', 'Monterrey', 'Tata', 'Tapas', 'Summer', 'Carr', 

stop_words9 = ['For', 'Design', 'And', 'With', 'While', 'During', 'Now', 'Business', 'Team', 'Telling', 'Week', 'Process', 'Working', 'Service', 'Health', 'Group', 'Such', 'Board', 
'Change', 'Office', 'Technical', 'Client', 'Market', 'Course', 'Yes', 'Model',  'Test', 'Getting', 'Mobile',  'Doing',  'Center', 'Building', 
 'College', 'National',  'Public',  'Below', 'Members', 'Learn','Store', 'Latin', 'Engineer',  'Chief',
 'Number',  'President', 'Problems', 'Section', 'Legal', 'Easy', 'Meeting', 'Check', 'Mexican', 'Women', 'Paper', 'Officer', 
 'Credit', 'Career', 'Cases', 'Session', 'Young', 'Tech', 'Chart', 'Subject', 'Private', 'Write', 'Startup',  'Scope', 'Action',
 
 'Korean', 'Indonesia', 'Machine', 'Execution', 'Policy', 'Money', 'Control', 'Above', 'Regional', 'Egypt', 'Special', 'Benefits', 'Café', 'Japanese', 'Council', 'Confidential',
'Japan', 'Reading', 'Turkey', 'Cause', 'White', 'Payment', 'Survey', 'Plant', 'Built', 'Know', 'Family', 'Operation', 'Hearing',  'Setting', 'Helps', 'Class', 'Schools',  'Clean', 
'Soldier', 'Columbia', 'Electric', 'Demand', 'Needs', 'Matrix', 'Access', 'Common', 'Search', 'Mother', 'Estate', 'Place', 'Five', 'Branch', 'Fashion', 'Space', 'Words', 'Talk', 'Trade',
'Nations', 'Scale', 'Certain', 'Walk', 'Theory', 'Labor', 'Electronic', 'Union', 'Payroll', 'Federal', 'Metro', 'Lower', 'Sports', 'Fuel', 'Magazine', 'Apps', 'Form', 'Calendar', 'Patent',
'Billing', 'Rate', 'Drilling', 'Rule', 'Points', 'Years', 'Contact', 'Secret', 'Chicago', 'Audio', 'Country', 'Music', 'Oracle', 'Rural', 'Italy', 'Supply', 'Asian', 'Finding',
'Reality', 'Travel', 'Simply', 'Waste', 'Gallery', 'Diary', 'Saudi', 'Brief', 'Father', 'Learner', 'Egyptian', 
'C', 'T', 'N', 'M', 'L',
'Roam', 'Bad', 'Windows', 'Purchase', 'Agent', 'Riding', 'Games', 'Unknown', 'Earth', 'Maximum', 'Tables',  'Old', 'Track', 'Orders', 'Army',
'Americas', 'Crazy', 'Player', 'Loyalty', 'Heads', 'Carrier', 'Palestine', 'Channels', 'Cancer', 'Auto', 'Chat', 'Stake', 'Contractor',  'Centers', 'Single', 'Venezuela', 'Navy',  'Guys',
 'Neither', 'Pricing', 'Trello', 'Domain', 'Stores', 'Cheap', 'Southern', 'Friends', 'Boards', 'Born', 'Canadian', 'Screen', 'Worth', 'Forum', 'Animal', 'Meal', 'Conflict',
 'Thailand', 'Sitting', 'Secretary', 'Blood', 'Character', 'Affinity', 'Female', 'Essay', 'Platinum', 'Turkish', 'Robot', 'Taken', 'Logic', 'Wife', 'Buyers', 'Click', 'Saving',
 'Weeks', 'Farmer', 'Weight', 'Ready', 'Distance', 'Sweden', 'Cup', 'Guardian', 'Sport', 'Turn', 'Freedom', 'Zealand', 'Sub',
 #2:
 'I', 'Insight', 'Learning', 'Finally', 'Development', 'Digital', 'B', 'Where', 'Its', 'Create', 'Making', 'His',
 #3:
#'Rehab', 'Dip', 
'Junior', 'West', 'Moore', 'Pink', 'Baker', 'Green', 'Gray', 'Brown',
# 'St.', 
 ]

stop_wordsU = ['Brazil', 'Mexico', 'Peru', 'September', 'Singapore', 'Australia', 'January', 'Pakistan', 'Nigeria', 'Korean', 'Indonesia', 'Latino', 'Italy', 
'Spain', 'Russia', 'Costa', 'France', 'Korea', 'British', 'Russian', 'Moscow', 'Thailand', 'Canada', 'Saturday', 'Lagos', 'Kenya',
'Chile', 'Ghana', 'Columbia', 'Delhi', 'Brazilian',
'Greece', 'London', 'Uber', 'Tuesday', 'Arabic', 'Mumbai', 'Latinos', 'Uganda', 'Arabia', 'Madrid', 'Feb', 'Aires', 'Rome', 'Dutch', 
'Americas', 'Persian', 'Mens', 'Amsterdam', 'Emirates', 'Sony', 'Berlin', 'Barcelona', 'Vietnam', 'Texas', 'Nov', 'Nigerian', 'Rican', 'Canadian', 'Harvard', 'Myanmar']
    
stop_words9M = ['Accepted', 'Advance', 'Aid', 'Aided', 'Aim', 'Algebra', 'Alliance', 'Aloe', 'Amethyst', 'Anchors', 'Arduino', 'Autumn', 
'Axis', 'Badge', 'Banana', 'Batch', 'Beach', 'Bear', 'Bench', 'Beverage', 'Bias', 'Bidding', 'Bio', 'Birth', 'Blanks', 'Block', 'Bold', 'Books', 'Booth', 
'Borders', 'Boss', 'Boy', 'Boys', 'Broaden', 'Broker', 'Broking', 'Bugs', 'Bulk', 'Bureau', 'Burns', 'Butterfly', 'Calm', 'Camera', 'Capability', 'Careers', 
'Carnival', 'Causing', 'Cell', 'Chairs', 'Champion', 'Charter', 'Chicken', 'Chosen', 'Church', 'Cinderella', 'Classified', 'Coast', 'Cold', 'Compact', 'Compass', 
'Convey', 'Corners', 'Corning', 'Corona', 'Count', 'Counting', 'Counts', 'Courage', 'Courier', 'Covert', 'Craft', 'Crisis', 'Crowd', 'Crystal', 'Cut', 'Dad', 
'Damper', 'Dear', 'Defer', 'Delay', 'Deputy', 'Dollar', 'Draft', 'Dream', 'Drew', 'Drivers',
'Drone', 'Dry', 'Dual', 'Dummy', 'Dust', 'Duty', 'Elite', 'Epic', 'Error', 'Evangelist', 'Evening', 'Eye', 'Factor', 'Failing', 'Farm', 'Fear', 'Fears', 'Fest', 'Fidelity', 'Figures',
'Fill', 'Filling', 'Film', 'Fine', 'Fire', 'Fiscal', 'Fish', 'Fit', 'Fitting', 'Fix', 'Flag', 'Flash', 'Flourish', 'Football', 'Former', 'Fort', 'Forty', 'Foster', 
'Fresh', 'Friend', 'Friendly', 'Fries', 'Funny', 'Galaxy', 'Gather', 'Gave', 'Gaze', 'Gentle', 'Geo', 'Giant', 'Gift', 'Gig', 'Girl', 'Glass', 'Globe', 'Gold', 'Gov', 
'Grab', 'Grammar', 'Greenhouse', 'Greet', 'Grip', 'Guard', 'Guess', 'Guru', 'Hall', 'Hands', 'Hardware', 'Heading', 'Healing', 'Heavy', 'Hey', 'Hires', 'Historical', 'Hobby', 
'Honesty', 'Hundred', 'Hunger', 'Hunting', 'Ice', 'Inception', 'Ink', 'Island', 'Jaguar', 'Jealousy', 'Joint', 'Judge', 'Jug', 'Justice', 'Kick', 'Kill', 'Kinda', 
'Label', 'Labour', 'Labs', 'Lamp', 'Land', 'Lately', 'Laws', 'Layer', 'Leather', 'Leaving', 'Led', 'Ledger', 'Left', 'Legend', 'Letters', 'Links', 'Losing', 'Lot', 'Lovers', 
'Macro', 'Magic', 'Mail', 'Mailer', 'Maker', 'Manger', 'Marking', 'Mask', 'Mass', 'Match', 'Matters', 'Mature', 'Meals', 'Meats', 'Memory', 'Memos', 'Mention', 'Messenger', 
'Meta', 'Meter', 'Middleman', 'Mile', 'Milestone', 'Milk', 'Mins', 'Modes', 'Mom', 'Moment', 'Monster', 'Mountain', 'Muslim', 'Must', 'Mystery', 'Mythology', 'Nail', 'Names',
'Necessary', 'Needle', 'News', 'Noble', 'Numbers', 'Older', 'Ones', 'Orange', 'Organized', 'Original', 'Orphan', 'Ours', 'Pages', 'Paint', 'Painting', 'Papers',  
'Party', 'Pass', 'Passion', 'Path', 'Pause', 'Paying', 'Peak', 'Pen', 'Penning', 'Persons', 'Pet', 'Piano', 'Pic', 'Picking', 'Piece', 'Pillars', 'Planes', 'Plants', 
'Polish', 'Poll', 'Pop', 'Ports', 'Poster', 'Pretty', 'Prime', 'Pro', 'Proto', 'Province', 'Queen', 'Query', 'Random', 'Raw', 'Rear', 'Reasons', 'Renew', 'Reps', 'Rest', 
'Reveal', 'Rift', 'Rise', 'Rising', 'Roads', 'Roles', 'Roof', 'Rope', 'Run', 'Rustic', 'Samples', 'Sat', 'Schooling', 'Seat', 'Seed', 'Seeds', 'Seems', 'Sell', 'Seller', 'Sellers',
'Selves', 'Send', 'Sense', 'Sensing', 'Sessions', 'Seven', 'Severe', 'Sexual', 'Shale', 'Shall', 'Sheets', 'Shorts', 'Signal', 'Silver', 'Sir', 'Sister', 'Slack', 'Sleep', 
'Slide', 'Smile', 'Soap', 'Solid', 'Spare', 'Speak', 'Spinning', 'Stack', 'Standing', 'Streets', 'Strength', 'Stuck', 'Style', 'Sunshine', 'Surprise', 'Sweets', 
'Tab', 'Tag', 'Tap', 'Taste', 'Teach', 'Tears', 'Teen', 'Threat', 'Throwing', 'Tier', 'Tittle', 'Tobacco', 'Topping', 'Touch', 'Tour', 'Toy', 'Tracker', 'Trader', 'Trading', 
'Transmission', 'Traveller', 'Trend', 'Truck', 'Truly', 'Trunk', 'Truth', 'Tuning', 'Turns', 'Twin', 'Upper', 'Us', 'Vegan', 'Venture', 'Vivid', 'Wait', 'Walking', 'Wanna', 'Want', 
'Warming', 'Warning', 'Wash', 'Wearing', 'Weekly', 'Wheat', 'Wheeler', 'Whistle', 'Wide', 'Wild', 'Wind', 'Wing', 'Winner', 'Winning', 'Wisdom', 'Worker', 'Worst', 'Yellow']
#'Apt',
#'Plenty', 'Paradise', 'Spark', 'Delta', 'Down', 'Sky', 

stop_wordsF = ['India', 'An', 'Can', 'Peer', 'Among', 'Lean', 'German', 'Loan', 'Made', 'Each']

stop_words10 = ['Colombia', 'Philippines', 'Inc', 'Bangladesh', 'Universidad', 'Netherlands', 'Dubai', 'Aramco', 'Starbucks', 'Confluence', 'México', 'Bogotá', 'Netflix', 
'Alzheimer', 'Symbian', 'Capgemini', 'Americans', 'Paraguay', 'Pixar', 'Australian', 'Airbnb', 'Unicamp']


#'No', 'David', 'Angela', 'Name', 'Africa', 'Virginia', 'Mars', 'May', 'John', 'Asia', 'Will', 'Ali', 'George', 'Ahmed', 'Martin', 'Art', 'Apple', 'Jose', 
#'Sunday', 'Princess', 'Steve', 'Geoff', 'April', 'Maria', 'June', 'Daniel', 'Ans', 'Ed', 'Anna', 'July', 'Andre', 'Tim', 'Tina', 'Ogilvie', 'Luis', 'Thomas', 'Kumar', 
#'Jeanne', 'Junior', 'Khan', 'Alice', 'Francisco', 'Lima', 'Hesse', 'Garcia', 'Lopez', 'Johnson', 'Marian', 'Carlos', 'Adam', 'Person', 'Jesus', 'Tony', 'Emily', 'Madeleine'
#'Petty', 'Antonio', 'Martinez', 'Michael', 'Car', 'Sam', 'Re', 'Sarkar', 'Smith', 'Vice', 'Gonzalez', 'Money', 'Tesla', 'Straw', 'Mr', 'Street', 

stop_wordsN7 = ['The', 'We', 'This', 'In', 'It', 'Application', 'Challenge', 'Selection', 'Approach', 'Insight', 'As', 'What', 'Mind', 'Visualization', 'For', 'After', 'Learning', 
    'Storytelling', 'Design', 'So', 'Our', 'Launch', 'To', 'Mapping', 'They', 'Thinking', 'Reflection', 'My', 'And', 'When', 'By', 'Then', 'If', 'With', 'There', 'But', 'One', 'At', 
    'However', 'These', 'Since', 'Also', 'Tool', 'While', 'How', 'Once', 'That', 'Each', 'During', 'All', 'From', 'Using', 'Now', 'Next', 'On', 'First', 'Innovation', 'Describe', 
    'Insights', 'Through', 'Therefore', 'Some', 'Business', 'Because', 'Example', 'He', 'Finally', 'Story', 'You', 'Most', 'India', 'Visual', 'People', 'Company', 'Many', 'Although', 
    'Management', 'Thus', 'Based', 'Hence', 'Assignment', 'She', 'Having', 'Even', 'Another', 'Customer', 'Map', 'Project', 'An', 'Before', 'Visualisation', 'Here', 'Every', 'Team', #An
    'Additionally', 'Given', 'Is', 'Data', 'University', 'Why', 'Marketing', 'Product', 'Development', 'Sometimes', 'Due', 'Moreover', 'Instead', 'Not', 'Being', 'Manager', 
    'Stories', 'Use', 'Journey', 'Telling', 'English', 'Furthermore', 'No', 'Which', 'Week', 'Process', 'New', 'Working', 'Future', 'Everyone', 'Module', 'Well', 'Value', 'Students', #No
    'Digital', 'More', 'Internet', 'Second', 'Do', 'Moving', 'Their', 'Department', 'User', 'According', 'Service', 'Human', 'Engineering', 'Final', 'Good', 'Of', 'Customers', 'Firstly', 
    'Those', 'Thanks', 'Google', 'Technology', 'Post', 'Key', 'Initially', 'Challenges', 'Secondly', 'Like', 'Facebook', 'Though', 'Often', 'Other', 'Scrum', 'Besides', 'Visio', 'Step', 
    'Just', 'School', 'Idea', 'Who', 'Research', 'System', 'Where', 'Time', 'Further', 'Problem', 'Applying', 'Overall', 'Currently', 'Indian', 'Recently', 'Within', 'Partners', 'Let', 
    'Health', 'Managers', 'Chain', 'Creating', 'App', 'Without', 'Finance', 'Following', 'Information', 'Unfortunately', 'Both', 'Maps', 'Make', 'Group', 'Later', 'Such', 'Coursera', 
    'Considering', 'Video', 'Your', 'Bank', 'Providers', 'Despite', 'Social', 'Board', 'Page', 'Its', 'Education', 'Agile', 'Instagram', 'Looking', 'Two', 'Reflection-', 'Create',  #Page
    'Brainstorming', 'Covid-19', 'Can', 'Going', 'Name', 'Experience', 'Last', 'Peer', 'Designing', 'Taking', 'Organizations', 'Brand', 'Ideas', 'Understanding', 'Broadly', 'Brazil', 
    'Today', 'Over', 'Connected', 'Training', 'Change', 'Internal', 'Feedback', 'Different', 'Quality', 'Participants', 'Communication', 'Tools', 'Organization', 'Only', 'General', 
    'Especially', 'Office', 'Making', 'Director', 'South', 'Upon', 'Technical', 'Basically', 'Her', 'Client', 'Usually', 'Are', 'Market', 'Course', 'Yes', 'Operations', 
    'Planning', 'Starting', 'High', 'Leadership', 'Me', 'Life', 'Employees', 'Model', 'Oversight', 'Congress', 'Be', 'Rather', 'His', 'Africa', 'Software', 'Or', 'Listening', 'Program',  #Africa
    'Empathy', 'Identify', 'Analysis', 'Government', 'Knowledge', 'Test', 'Launches', 'Cost', 'Power', 'Along', 'Several', 'Would', 'Thank', 'Senior', 'Corporate', 'Maybe', 'Growth', 
    'Industry', 'Insurance', 'Head', 'Institute', 'Questions', 'Analytics', 'Global', 'City', 'State', 'Myer', 'United', 'Phase', 'Lastly', 'Mexico', 'Big', 'Financial', 'Have', 'Getting', #Myer
    'Online', 'Start', 'Services', 'Figure', 'America', 'Apart', 'Testing', 'Case', 'Works', 'Resources', 'Lack', 'Science', 'Third', 'Kitchen', 'Microsoft', 'Plan', 'Knowing', 'Mobile', #Case
    'Throughout', 'Define', 'Europe', 'Clearly', 'Similarly', 'Any', 'Clients', 'Ultimately', 'Among', 'Doing', 'Perhaps', 'Designers', 'Stage', 'Regarding', 'Afterwards',
    'Work', 'Executive', 'Visualizing', 'Generally', 'Strategy', 'Seeing', 'Take', 'Support', 'Day', 'Whenever', 'Wows', 'Stakeholders', 'Personas', 'Nevertheless', 'Solution', 'Again', 
    'Consequently', 'Performance', 'African', 'Center', 'Concept', 'Banking', 'Part', 'Risk', 'Together', 'Indeed', 'Children', 'Non', 'Does', 'Yet', 'Reﬂection', 'Building', 'Did', 'Meyer', #Meyer Did
    'Specifically', 'Conclusion', 'Transformation', 'College', 'Co', 'Focus', 'Right', 'Keeping', 'Under', 'National', 'Smart', 'Prototype', 'Introduction', 'Teams', 'Personal', 'International', 
    'Something', 'Creative', 'Computer', 'Users', 'American', 'Personally', 'Current', 'Drawing', 'European', 'Sharing', 'Care', 'Peru', 'Point', 'Few', 'Out', 'Consumer', 'Colombia', #Few
    'Eventually', 'Virtual', 'Exercise', 'Think', 'Build', 'Nowadays', 'Cloud', 'Zoom', 'Covid', 'Criteria', 'Teachers', 'Better', 'March', 'Everything', 'Note', 'States', 
    'Things', 'End', 'Master', 'Delivery', 'Explain', 'Actually', 'Hypothesis', 'Whether', 'Develop', 'Main', 'Below', 'China', 'Web', 'Public', 'Parents', 'Solutions', 'Consultant',
    'Date', 'Members', 'Lead', 'Excel', 'Core', 'Everybody', 'Liedtka', 'Production', 'Results', 'Administration', 'Until', 'Media', 'Level', 'Very', 'Workshop', 'About', 'Chinese', #Liedka
    'Prior', 'Apple', 'Likewise', 'Real', 'Self', 'Emotional', 'Please', 'Platform', 'East', 'Small', 'Impact', 'September', 'Get', 'Elements', 'Spanish', 'Singapore', 'Diagram', 'Vision', #Apple
    'Ministry', 'Canvas', 'World', 'Pictures', 'Learn', 'Wow', 'Store', 'Three', 'Ask', 'Back', 'October', 'Inc.', 'Latin', 'Food', 'Previously', 'Simple', 'Industrial', 'Ideation', 'Identifying', 
    'Australia', 'December', 'Add', 'Depending', 'Creation', 'Subsequently', 'Lean', 'Prototyping', 'Teaching', 'I´m', 'Mindmapping', 'January', 'Central', 'Engineer', 'Improve', 'Go', 'Enterprise', 
    'Launching', 'Medical', 'Applications', 'Pakistan', 'Perspective', 'Red', 'Manufacturing', 'Intelligence', 'Chief', 'Background', 'Nigeria', 'Resource', 'Understand', 'Miro', # Red Miro
    'Korean', 'Machine', 'Task', 'Employee', 'Keep', 'Developing', 'Increase', 'Number', 'Staff', 'Had', 'Statement', 'Assumptions', 'Soon', 'Directors', 'President', 'Problems', 'Section', #Had Soon
    'Interviewee', 'Professional', 'Designer', 'Strategic', 'Coming', 'Indonesia', 'Easy', 'Latino', 'Amazon', 'Event', 'Analyst', 'Peace', 'Healthcare', 'Meeting', 'Review', 'Security', 
    'Unit', 'Check', 'Solving', 'Centre', 'North', 'Earlier', 'Live', 'Legal', 'Interviews', 'Consumers', 'Home', 'French', 'Need', 'Companies', 'Persona', 'Question', 'Mexican', #Moore
    'November', 'Safety', 'See', 'Obviously', 'Women', 'York', 'Almost', 'Imagine', 'Graded', 'Italy', 'Additional', 'Trying', 'Flow', 'Shared', 'Giving', 'Paper', 'Generation', 'Pre', #----
    'Best', 'Supply', 'Put', 'Unlike', 'Committee', 'Category', 'Partner', 'Account', 'Credit', 'Adding', 'Career', 'References', 'Educational', 'Price', 'Author', 'Sprint', 'Was', 'Fast', 
    'Implementation', 'Providing', 'Projects', 'Products', 'Job', 'Systems', 'Images', 'Retail', 'Still', 'Officer', 'Normally', 'Farmers', 'Content', 'Experts', 'Study', 'Argentina',  #Job
    'Multiple', 'Play', 'Lego', 'Find', 'Open', 'Picture', 'Luckily', 'Infrastructure', 'Field', 'Cases', 'Spain', 'Session', 'Fortunately', 'Description', 'Reference', 'Share', 'Examples', 
    'Could', 'Improvement', 'Youth', 'German', 'Little', 'Oil', 'Free', 'August', 'Great', 'Tech', 'Leaders', 'Interface', 'Target', 'Philippines', 'Whilst', 'Loan', 'Telecom', 
    'Submitted', 'Bringing', 'Consulting', 'Similar', 'Presenting', 'Minimum', 'Asian', 'Should', 'Graphic', 'Russia', 'Foundation', 'Finding', 'Germany', 'Same', 'Discussion', 'Relationship', 
    'Coffee', 'Organizational', 'Chart', 'Network', 'Pilot', 'Database', 'Subject', 'Talent', 'Private', 'Meanwhile', 'Surprisingly', 'Mechanical', 'Notes', 'Automation', 'Give', 'Selected', 
    'Various', 'Alternative', 'Distribution', 'Logistics', 'Shop', 'Community', 'Hiring', 'Individual', 'Interviewer', 'Write', 'Goals', 'Always', 'Typically', 'Reality', 'Club', 'Middle', 
    'Thereafter', 'Hyderabad', 'Ideate', 'Lab', 'Activities', 'Studies', 'Voice', 'Workshops', 'Agriculture', 'Italian', 'Engagement', 'Around', 'Startup', 'Requirements', 'Investment', 
    'Twitter', 'Culture', 'Act', 'Game', 'Plus', 'Artificial', 'Scope', 'Compliance', 'Friday', 'Structure', 'Up', 'Gas', 'Segment', 'Rubric', 'Brands', 'Communications', 'Title', 'Top', 
    'Existing', 'Area', 'Action', 'Major', 'Effective', 'Wi', 'Fi', 'Presentation', 'Air', 'Word', 'Activity', 'Provide', 'Thirdly', 'Six', 'Stakeholder', 'Draw', 'Travel', 'Saudi', 
    'Email', 'Classification', 'Banks', 'Procurement', 'Issues', 'France', 'Simply', 'Method', 'Revenue', 'Physical', 'Programme', 'List', 'Waste', 'Failure', 'Putting', 'Korea', 'Image', 
    'Energy', 'Otherwise', 'Electronics', 'Campus', 'Specific', 'Scientific', 'Decision', 'British', 'Skype', 'Maintenance', 'Beyond', 'Interestingly', 'Engaging', 'Annual', 'Long', 'Visualizations', 
    'Show', 'None', 'Environment', 'Fig', 'Root', 'Net', 'Operational', 'Goal', 'Consultants', 'Conducting', 'Monday', 'Industries', 'Present', 'Steps', 'Language', 'Audience', 'Gallery', 'Try', 
    'Improving', 'Set', 'E.g.', 'Diary', 'Kanban', 'Priority', 'Creativity', 'Showing', 'Faculty', 'Age', 'Select', 'Managing', 'Architecture', 'Majority', 'Russian', 'Medium', 'Execution', 
    'Material', 'Ideally', 'Operating', 'Asking', 'Implementing', 'Result', 'Engineers', 'Moscow', 'Policy', 'Patients', 'Scenario', 'Satisfaction', 'Commercial', 'Money', 'Budget', 'Limited', 
    'Control', 'Tableau', 'Inc', 'Assumption', 'Straw', 'Institutions', 'Regional', 'Much', 'Writing', 'Allow', 'Made', 'Greek', 'Success', 'February', 'Above', 'Secondary', 'Pin', 'Processing', 
    'Important', 'Sector', 'Expert', 'Recruitment', 'Egypt', 'Mostly', 'Christmas', 'Special', 'Benefits', 'Accordingly', 'Alternatively', 'Help', 'Others', 'Tell', 'Water', 'Ethnography', 
    'Line', 'Selling', 'Talking', 'Viable', 'Requirement', 'Approach-', 'Café', 'Traditional', 'Used', 'Person', 'Japanese', 'Mindset', 'Reason', 'Cross', 'Organisation', 'The\u200b', 
    'Urban', 'Essentially', 'Academic', 'Council', 'Report', 'Less', 'Clinic', 'Rapid', 'Excellence', 'Request', 'Four', 'Ethnographic', 'Confidential', 'Changing', 'Traditionally',
    'Innovative', 'Visualize', 'Table', 'Selection-', 'Application-', 'Planners', 'Japan', 'External', 'Thailand', 'Diagrams', 'Canada', 'Acquisition', 'Initial', 'Coaching',
    'Choose', 'Pandemic', 'Executives', 'Reading', 'Agents', 'Skills', 'Agency', 'Local', 'Coronavirus', 'Too', 'Turkey', 'Cause', 'Payment', 'Order', 'Saturday', 'Collaboration',
    'Assessment', 'Association', 'Survey', 'Automotive', 'Objective', 'Role', 'Proposition', 'Mental', 'Outcome', 'Discovery', 'Jamboard', 'Entrepreneurship', 'Kenya', 'Plant', 'Portal', 
    'Groups', 'Lunch', 'Remember', 'Built', 'Nonetheless', 'Involving', 'Higher', 'Matter', 'Know', 'Individuals', 'Created', 'Critical', 'Short', 'Interactive', 'Submission', 'Direct', 
    'Videos', 'Needless', 'Academy', 'Accounting', 'Definitely', 'Sustainable', 'Vendor', 'Sciences', 'Whatever', 'Integration', 'Whereas', 'Suddenly', 'Leading', 'Defining', 
    'Family', 'Focusing', 'Started', 'Gathering', 'Identified', 'Call', 'Covid19', 'Bangladesh', 'Workplace', 'Cyber', 'Increased', 'I´ve', 'Operation', 'Pension', 'Context', 'Hearing', 
    'Evaluation', 'Including', 'Applied', 'Materials', 'Chile', 'Avoid', 'Ecuador', 'Society', 'Asset', 'Augmented', 'Issue', 'Large', 'Continuous', 'Force', 'Assistant', 'Governance', 
    'Mural', 'Standard', 'Positive', 'Child', 'Card', 'Methodology', 'Setting', 'Honestly', 'Sometime', 'Pharmaceutical', 'Consider', 'Blue', 'Competition', 'Helps', 'Reflecting', 'Ghana', 
    'Library', 'Sourcing', 'Class', 'Zone', 'Schools', 'Read', 'Multi', 'Hopefully', 'Exploration', 'Onboarding', 'Complex', 'Universidad', 'Room', 'Bachelor', 'Available', 'Meetings', 
    'Firm', 'Reflections', 'Trust', 'Shortly', 'Fortune', 'Collecting', 'Transport', 'Sale', 'Index', 'Clean', 'Explaining', 'Methods', 'Purpose', 'Solar', 'Netherlands', 'Awareness', 
    'Aside', 'Usage', 'Kids', 'Selecting', 'Studying', 'Interview', 'Python', 'Ensure', 'Funds', 'Soldier', 'Columbia', 'Adobe', 'Illustrator', 'Implement', 'Electric', 'Demand', 'Score', 
    'Efficiency', 'Half', 'Approval', 'Needs', 'Western', 'Android', 'Soft', 'Matrix', 'Book', 'Associate', 'Events', 'Dubai', 'Aramco', 'Offer', 'Changes', 'Access', 'Supplier', 'Common', 
    
    'Primary', 'Eastern', 'Coach', 'Choosing', 'Conduct', 'Branches', 'Technique', 'Helping', 'History', 'Ex', 'Combining', 'Search', 'Reduce', 'Mother', 'Speed', 'Physics', 'Doctors', 
    'Region', 'Responsibility', 'Grade', 'Learnings', 'Capital', 'Stress', 'Advertising', 'Establishing', 'Black', 'Delhi', 'Front', 'Brazilian', 'Greece', 'Face', 'Tree', 'Professionals', 
    'Sustainability', 'Communicating', 'Radio', 'Estate', 'Starbucks', 'Serious', 'Fundraiser', 'Presentations', 'Topic', 'Establish', 'Hello', 'Youtube', 'Sketching', 'Discover', 'Laundry', 
    'Reports', 'Humans', 'Techniques', 'Document', 'Departments', 'Early', 'Active', 'Sample', 'Basis', 'Already', 'Place', 'Till', 'London', 'Five', 'Experimentation', 'Immediately', 'Branch', 
    'Housing', 'Designs', 'Construction', 'Observation', 'Pitch', 'Look', 'Carbon', 'Gender', 'Return', 'Fashion', 'Lessons', 'Space', 'Words', 'Naturally', 'Specially', 'Uber', 'Probably', 
    'Confluence', 'Talk', 'Increasing', 'Member', 'Year', 'Divine', 'Manual', 'Connecting', 'Uruguay', 'Leads', 'Previous', 'Reporting', 'Campaign', 'Particularly', 'Advantages', 'Trade', 
    'Napkin', 'Opportunities', 'Prototypes', 'Analyzing', 'Drive', 'Press', 'Sub', 'Allowing', 'Photoshop', 'Storyboard', 'Answers', 'Kingdom', 'Between', 'Conference', 'Equipment', 'Proposal', 
    'Switzerland', 'Indicators', 'Nations', 'Airport', 'Objectives', 'Gaining', 'Division', 'Compared', 'Scale', 'Motivation', 'Shopping', 'Website', 'Answer', 'Clear', 'Patient', 'Rules', 
    'Balance', 'Vehicle', 'True', 'Reduction', 'Engine', 'Maersk', 'Approaches', 'Certain', 'Walk', 'Opening', 'Hotel', 'Related', 'Interaction', 'Challange', 'Everyday', 'Poor', 'Situation', 
    'Schedule', 'Mainly', 'Nothing', 'Theory', 'Hospital', 'Fail', 'Buyer', 'Type', 'Interior', 'Device', 'Principal', 'Portfolio', 'Promoter', 'Agencies', 'Nature', 'Labor', 'Costs', 'México', 
    'Preparing', 'Electronic', 'Union', 'Average', 'Desing', 'Natural', 'Done', 'Quito', 'Logo', 'Payroll', 'Slowly', 'Invite', 'Advanced', 'Identification', 'Cards', 'Win', 'Enterprises', 
    'Tuesday', 'Malaysia', 'Park', 'Einstein', 'Literacy', 'Basic', 'Brainstorm', 'Generate', 'Engage', 'Thereby', 'Resolution', 'Anniversary', 'Buy', 'Studio', 'Developer', 'Movie', 'Corporation', 
    'Lecture', 'Were', 'Federal', 'Quite', 'District', 'Past', 'Plastic', 'Break', 'Metro', 'Framework', 'Color', 'Breaking', 'Experiences', 'Facility', 'Anyway', 'Lots', 'Lower', 'Charts', 
    'Definition', 'Gain', 'Quick', 'Sports', 'Emergency', 'Math', 'Musk', 'Dev', 'Fuel', 'Magazine', 'Tests', 'Application:-', 'Apps', 'Incubator', 'Really', 'Speaking', 'Form', 'Masters', 
    'Town', 'Remote', 'Analyze', 'Facing', 'Exam', 'Total', 'Juniors', 'Calendar', 'Wireless', 'Headphones', 'Robotics', 'Molybdenum', 'Graphics', 'Bengali', 'Modern', 'Economy', 
    'Relations', 'Months', 'Experiment', 'Patent', 'Property', 'Values', 'Record', 'Technological', 'Exchange', 'Perfect', 'Exploring', 'Electrical','Prepared', 'Normal', 'Continuing', 'Output', 
    'Coordinator', 'Site', 'Developers', 'Billing', 'Rate', 'Optimization', 'Retailers', 'Inspired', 'Photography', 'Bogotá', 'Coca', 'Cola', 'Elderly', 'Parks', 'Drilling', 'Spending', 
    'Location', 'Rule', 'View', 'Birthday', 'Wealth', 'Points', 'Monthly', 'Provider', 'Functional', 'Unique', 'Years', 'Mathematics', 'Statistics', 'Contact', 'Gradually', 'Recent', 'Organic', 
    'Alumni', 'Apply', 'Volunteers', 'Toastmasters', 'Degree', 'Chemical', 'Myself', 'Measure', 'Netflix', 'Secret', 'Anxiety', 'Operators', 'Incorporating', 'Cybersecurity', 'Narrative', 
    'Associates', 'Publishing', 'Accounts', 'Traffic', 'Selection:-', 'Packaging', 'Possible', 'Follow', 'Release', 'Might', 'Input', 'Addressing', 'Commerce', 'Include', 'Chicago', 'Features', 
    'Source', 'Circle', 'Internally', 'Audio', 'Arabic', 'Admin', 'Reviewing', 'Projective', 'Discussing', 'Specialist', 'Visualising', 'Wellness', 'Mumbai', 'Carriers', 'Bitcoin', 'Country', 
    'Promoting', 'Transfer', 'Income', 'Maintain', 'Introducing', 'Municipality', 'Music', 'Parts', 'Profit', 'Provided', 'Counseling', 'Architect', 'Inventory', 'Journal', 'Manage', 'Response', 
    'Guide', 'Improved', 'Eliminate', 'Rural', 'Stream', 'Latinos', 'Advantage', 'Tax', 'Brief', 'Inputs', 'Facilities', 'Employer', 'Agreement', 'Father', 'Whole', 'Semester', 'Moderate', 'Discuss', 
    'Full', 'Proof', 'Box', 'Discussions', 'Learners', 'Administrative', 'Northern', 'Resolutions', 'Learner', 'Recovery', 'Apparently', 'Plot', 'Promote', 'Mission', 'Visit', 'Uganda', 'Etc', 
    'Prevention', 'Alzheimer', 'Leader', 'Peruvian', 'Egyptian', 'Ambassador', 'Patterns', 'Sensitivity', 'Double', 'House', 'Assistants', 'Leaning', 'Branding', 'Salesforce', 'Clinical', 'I`m', 
    'Indians', 'Challenge-', 'Symbian', 'Population', 'Meaning', 'Chapter', 'Originally','Equity', 'Analysts', 'Regardless', 'Historically', 'Jira', 'Workflow', 'Processes', 'Admissions', 
    'Pressure', 'Collaborative', 'Efficient', 'Entrepreneur', 'Technologies', 'Iran', 'Environmental', 'Windows', 'Drug', 'Confirming', 'Org', 'Identity', 'Micro', 'Eg', 'Purchase', 'Classroom', 
    'Vendors', 'Meet', 'Interesting', 'Closing', 'Agent', 'Institution', 'Capgemini', 'Unless', 'Restaurant', 'Demo', 'Elementary', 'Oblasts', 'Lately', 'Economics', 'Loss', 'Practical', 'Decide', 
    'Observing', 'Riding', 'Tracking', 'Stock', 'Practice', 'Monitoring', 'Typical', 'Renaissance', 'Games', 'Link', 'Founder', 'Unknown', 'Written', 'Electricity', 'Universities', 'Owner', 'Train', 
    'Arabia', 'Kinect', 'Volume', 'Challenge:\u200b', 'Arab', 'Retrieved', 'Documentation', 'Strong', 'Earth', 'Madrid', 'Americans', 'Artists', 'Utilizing', 'Availability', 'Azerbaijan', 'Added', 
    'Sponsorship', 'Separate', 'Entrepreneurs', 'Crafting', 'Outcomes', 'Pharma', 'Terms', 'Feb', 'Provides', 'Actual', 'Transaction', 'Station', 'Performing', 'Narrating', 'Frequently', 'Maximum', 
    'Induction', 'Potential', 'Integrated', 'Mindmap', 'Print', 'Powerpoint', 'Aires', 'Humidity', 'Tables', 'Whiteboard', 'Representatives', 'Classes', 'Amongst', 'Either', 'Triple', 'Old', 'Paraguay', 
    'Track', 'Advisor', 'Inside', 'Assembly', 'Round', 'Inspiration', 'Layout', 'Stick', 'Organize', 'Afterward', 'Daily', 'Proper', 'Lesson', 'Pillar', 'Alternatives', 'Somehow', 'Approach:-', 'Sponsors', 
    'Storyboarding', 'Fourth', 'Facts', 'Workers', 'Organizing', 'Orders', 'Oral', 'Army', 'Psychology', 'Capture', 'Visuals', 'Empathize', 'Collection', 'Complete', 'Businesses', 'Innovate', 'Focused', 
    'Turning', 'Fintech', 'Domain', 'Framing', 'It´s', 'Hospitals', 'Theater', 'Lockdown', 'Bring', 'Principles', 'Desk', 'Americas', 'Crazy', 'Colleagues', 'Aplication', 'Encourage', 'Improves', 
    'Civil', 'Never', 'Citizens', 'Domestic', 'Carrier', 'Palestine', 'Cycles', 'Detail', 'Assurance', 'Retention', 'Persian', 'Mens', 'Player', 'Amsterdam', 'Battery', 'Competitors', 'Running', 'Sadly', 
    'Representative', 'Confidence', 'Prioritization', 'Networks', 'Observe', 'Adventure', 'Cultural', 'Experimenting', 'Strategies', 'Areas', 'Equally', 'Anyone', 
           
    'Advisory', 'Application\u200b', 'Challenge\u200b', 'We\u200b', 'Insight\u200b', 'Automated', 'Cycle', 'Pick', 'Climate', 'Emirates', 'Sticky', 'Outsourcing', 'Code', 'Govt', 'Profession', 'Disadvantage', 
    'Adapt', 'Progress', 'Collective', 'Ecosystem', 'Garden', 'Detailed', 'Political', 'Collected', 'Happiness', 'Sony', 'Millennial', 'Towards', 'Concepts', 'Medicine', 'Trello', 'Organisational', 'Channels', 
    'Repository', 'Listen', 'Alternate', 'Fixed', 'Repeat', 'Fishbone', 'Surveys', 'Briefly', 'Initiative', 'Construct', 'Buenos', 'Undoubtedly', 'Economic', 'Prepare', 'Suite', 'Cancer', 'Audiences', 
    'Attendees', 'Networking', 'Relevant', 'Inspiring', 'Berlin', 'Innovator', 'Excellent', 'Amount', 'Size', 'Transparency', 'Direction', 'Regulatory', 'Begin', 'Entertainment', 'Printing', 'Regular', 'Disease', 
    'Simplify', 'Curriculum', 'Insight-', 'Structural', 'Sponsor', 'Memorial', 'Option', 'Importance', 'Beside', 'Hackathon', 'Living', 'Text', 'Representation', 'Documents', 'Forward', 'Contractor', 'Describing', 
    'Continuity', 'Occasionally', 'Trainings', 'Ladies', 'Factors', 'Neither', 'Centers', 'Diversity', 'Targets', 'Contribution', 'Diabetes', 'Single', 'Venezuela', 'Disconfirming', 'Unicamp', 'Namibia', 'Debit', 
    'Wireframes', 'Mortgage', 'Period', 'Types', 'Primarily', 'Skill', 'Congressional', 'Collect', 'Backlog', 'Taiwan', 'Sorry', 'Item', 'Realtors', 'Brussels', 'Guys', 'Adult', 'Staying', 'Details', 
    'Bike', 'Pricing', 'Marketplace', 'Introduce', 'Stores', 'Risks', 'Instructions', 'Courses', 'Phone', 'Preparation', 'Refection', 'Programs', 'Mirror', 'Tourism', 'Pride', 'Questionnaire', 'Trainer', 'Days', 
    'Forms', 'Hospitality', 'Approach:\u200b', 'Vietnam', 'Agreements', 'Custom', 'Suppliers', 'Appraisal', 'Methodologies', 'Finances', 'I´ll', 'Nobody', 'Southern', 'Teamwork', 'Reverse', 'Friends', 
    'Attractive', 'Boards', 'Searching', 'Loyalty', 'Compare', 'Consistent', 'Timeline', 'Condition', 'Regulation', 'Millennials', 'Biological', 'Nigerian', 'Intelligent', 'Experiments', 'Investors', 'Sessions', 
    'Segments', 'Centered', 'Handling', 'Disaster', 'Productivity', 'Internship', 'Synthesis', 'Ability', 'Relationships', 'Surely', 'Beginning', 'Conflict', 'Incident', 'Assuming', 'Certainly', 'Manufacturer', 
    'Premium', 'Procedure', 'Blind', 'Light', 'Followed', 'Display', 'Redesigning', 'Dynamic', 'Graph', 'Authenticity', 'Quickly', 'Citizen', 'Graphs', 'Tourists', 'Timely', 'Infographics', 'Animal', 'Competencies', 
    'Anything', 'Insight:-', 'Buying', 'Forum', 'Opinion', 'Animation', 'Address', 'Requests', 'Distributor', 'Restricted', 'Conducted', 'Character', 'Defined', 'Pixar', 'Items', 'Girls', 'Challenge:-', 'Gentlemen', 
    'Pattern', 'Deliver', 'Australian', 'Syllabus', 'Successful', 'Thankfully', 'Hypotheses', 'Inspire', 'Template', 'Landscape', 'Summary', 'Discussed', 'Except', 'Ensuring', 'Numerous', 'Simultaneously', 'Offering', 
    'Innovations', 'Canadian', 'Perception', 'Myanmar', 'Airbnb', 'Essential', 'Launch-', 'Ambiguity', 'Continue', 'Reaction', 'Suppose', 'Purchasing', 'Satellite', 'Guinness', 'Effects', 'Architectural', 'Deutsche', 
    'Storyboards', 'Festival', 'Probability', 'Photo', 'Feature', 'Belonging', 'Century', 'Flipped', 'Visualizaton', 'Fundraising', 'Improvements', 'Sources', 'Regards', 'Scheduling', 'Opportunity', 'Upgrading',
    'Profiling', 'Candidates', 'Slides', 'Flexibility', 'Presently', 'Descriptive', 'Appendix', 'Ancient', 'Opinions', 'Actions', 'Eating', 'Cooking', 'Collectively', 'Shift', 'Bluetooth', 'Perú', 'Honduras', 
    'Reinforce', 'Laboratory', 'Screen', 'Procedures', 'Highly', 'Proposed', 'Workforce', 'Audit', 'Champions', 'Linked', 'Toolkit', 'Toronto', 'Religion', 'Affinity', 'Leave', 'Accessed', 'Paragraph', 'Contracting', 
    'Alliance', 'Persons', 'Combine', 'Minimize', 'Enhancing', 'Thesis', 'Growing', 'Comfortable', 'Calculate', 'Sheets', 'Interest', 'Walmart', 'Parking', 'Recognition', 'Paying', 'Highlighting', 'Package', 
    'Leveraging', 'Constant', 'Effect', 'Welcome', 'Personality', 'Maping', 'Sitting', 'Translate', 'Aproach', 'Models', 'Practically', 'Gather', 'Hours', 'Mobility', 'Walking', 'Counselling', 'Qualitative', 'Scientist', 
    'Levels', 'Server', 'Topics', 'Easter', 'Filipinos', 'Warehouse', 'Accenture', 'Secretary', 'Expected', 'Approximately', 'Employment', 'Guatemala', 'Diverse', 'Turkish', 'Workspace', 'Presence', 'Barriers',
    'Validation', 'Interviewing', 'Standardization', 'Linear', 'Graphical', 'Namely', 'Generating', 'Village', 'Empathise', 'Explanation', 'Temp', 'Frequent', 'Dandruff', 'Untitled', 'Lionsgate', 'Quantitative', 
    'Mining', 'Scenarios', 'Learnt', 'Concurrently', 'Airlines', 'Colombian', 'Faster', 'Intellectual', 'Wherever', 'Narration', 'Referring', 'Connections', 'Female', 'Quiz', 'Significant', 'Strengths', 
    'Photographs', 'Researching', 'Jogging', 'Leisure', 'Attention', 'Nearly', 'Possibly', 'Evaluating', 'Computing', 'Connect', 'Caregivers', 'Possibilities', 'Observations', 'Periodic', 'Museum', 
    'Glucose', 'Reduced', 'Practices', 'Robot', 'Meteorological', 'Smartphone', 'Flexible', 'Empower', 'Ukraine', 'Upcoming', 'Ecobrick', 'Bundle', 'Believe', 'Statements', 'Guest', 'Selection\u200b', 'Trends', 
    'Millions', 'Prepaid', 'Distributors', 'Compact', 'Coupled', 'Defence', 'Faced', 'Enabling', 'Investing', 'Importantly', 'Complexity', 'Enough', 'Elevator', 'Deposit', 'Thought', 'Encourages', 'Speaker', 
    'Parent', 'Accessibility', 'Promotion', 'Tasks', 'Lebanon', 'Raising', 'Journeys', 'Countries', 'Prefect', 'Racing', 'Geodesic', 'Untold', 'Soldiers', 'Pharmacy', 'Essay', 'Charity', 'Artist', 'Phases', 
    'Temporary', 'Computers', 'Organized', 'Smartphones', 'Gained', 'Differentiation', 'Visually', 'Analytical', 'Telecommunications', 'Africans', 'Respect', 'Duration', 'Durable', 'Length', 'Feedbacks', 
    'Participation', 'Constantly', 'Informal', 'Formal', 'Helpdesk', 'Regularly', 'Presented', 'Lecturer', 'Missing', 'Obstacles', 'Counselor', 'Individually', 'Chairman', 'Drawings', 'Scaling', 'Modelling', 
    'Revolution', 'Selection:\u200b', 'Desktop', 'Visualized', 'Institutional', 'Payments', 'Competitive', 'Savings', 'Visibility', 'Decisions', 'Sprints', 'Requires', 'Affordable', 'Application:\u200b', 'Mentor', 
    'Incorrect', 'Facilitator', 'Behavioural', 'Sanitation', 'Librarians', 'Shipping', 'Someone', 'Functions', 'Outline', 'Ltd', 'Visits', 'Logic', 'Petrol', 'Pizza', 'Boomers', 'Therefor', 'Buyers', 'Certificate', 
    'Crucial', 'Digging', 'Programming', 'Responsible', 'Bucket', 'Weight', 'Interactions', 'Click', 'Therapists', 'Advertisement', 'Boarding', 'Wordpress', 'Desirability', 'Viability', 'Robotic', 'Rising', 'Tried', 
    'Servers', 'Certification', 'Saving', 'Steering', 'Secure', 'Literature', 'Grocery', 'Clusters', 'Guided', 'Hydroponics', 'I’am', 'Operator', 'Volunteer', 'Export', 'Utility', 'Motivations', 'Responses', 
    'Centric', 'Distance', 'Sweden', 'Locality', 'Belgium', 'Transportation', 'Interpretation', 'Marketers', 'Mapped', 'Letter', 'Physicians', 'Videocall', 'Realization', 'Relevance', 'Dashboards', 'Textile', 'Minimal', 
    'Incubation', 'Multinational', 'Anywhere', 'Selfcare', 'Volkswagen', 'Eliminating', 'Redesign', 'Thoughts', 'Surcharge', 'Parties', 'Express', 'Conversion', 'Employing', 'Oftentimes', 'Negative', 'Algorithm', 
    'I`ve', 'Desired', 'Overview', 'Facilitating', 'Deployment', 'Intern', 'Standing', 'Adopting', 'Scalability', 'Childhood', 'Effectively', 'Petroleum', 'Recommendations', 'Comparing', 'Worker', 'Cooperation', 
    'Remembering', 'Investigate', 'Painting', 'Assignment-', 'Approach\u200b', 'Designed', 'Frankly', 'Coordination', 'Random', 
#                'I', 'A', 'Mr', 'Student'
    'Thinks', 'Protocol', 'Hindu', 'Evolution', 'Pyramid', 'Financing', 'Docs', 'Status', 'Says', 'Land', 'Certified', 'Gym', 'Linux', 'Hear', 'Tier', 'Plastics', 'Series', 'Farm', 'Islamic', 'Hands', 'Authority', 
    'Labs', 'Odyssey', 'Contract', 'Enjoy', 'Path', 'Vol', 'Dear', 'Raspberry', 'Norwegian', 'Capacity', 'Ramadan', 'Washington', 'Thinkers', 'Minister', 'Behind', 'Hold', 'Foreign', 'Tabs', 'Tv', 'Ankara', 'Truly', 
    'Historical', 'Seafood', 'Jewellery', 'Winner', 'Advance', 'Provision', 'Qtr', 'Poland', 'Known', 'England', 'Seminar', 'Ground', 'Ring', 'Aiming', 'Sign', 'Gaming', 'Send', 'Filter', 'Diet', 'Evidence', 
    'Forbes', 'Chamber', 'Denmark', 'Aids', 'Coaches', 'Unified', 'Church', 'Guidance', 'Invest', 'Tanzania', 'Temperature', 'Names', 'Us', 'Sounds', 'Weekend', 'Pivot', 'Version', 'Swiss', 'Factor', 'Cooperative', 
    'Tribal', 'Belgian', 'Arduino', 'Finland', 'Balloons', 'Iron', 'Landing', 'Speech', 'Dream', 'Tour', 'Specialists', 'Humanity', 'Difficult', 'Editorial', 'Sketches', 'Logistic', 'Rotterdam', 'Bayer', 'Malta', 
    'Benefit', 'Diploma', 'Million', 'Ericsson', 'Junk', 'Thinker', 'Muslim', 'Seller', 'Style', 'Traveller', 'Pass', 'Skin', 'Bear', 'Guess', 'Saved', 'Pages', 'Twice', 'Wrong', 'Captain', 'Retailer', 'Austria', 
    'Upper', 'Memory', 'Caribbean', 'Roaming', 'Nurses', 'Male', 'Timing', 'Georgian', 'Prospect', 'Depression', 'Rowling', 'Kindergarten', 'Fears', 'Blessing', 'Cotton', 'Waterfall', 'Habitat', 'Retro', 'Aware', 
    'Bags', 'Milton', 'Margin', 'Newspaper', 'Comfort', 'Lisbon', 'Portugal', 'Spotify', 'Alexandria', 'Minor', 'Tokyo', 'Kindly', 'Bolivia', 'Greater', 'Flight', 'Limit', 'Buffet', 'Nacional', 'Israel', 'Closed', 
    'Cambridge', 'Bias', 'Valenzuela', 'Alibaba', 'Wild', 'Batch', 'Occasion', 'Guaranteed', 'Honest', 'Ordinary', 'Mutual', 'Kings', 'Block', 'Blank', 'Vertical', 'Danish', 'Shanghai', 'Scatter', 'Exhibit', 
    'Night', 'Programmer', 'Camera', 'Adriatic', 'Planet', 'Battle', 'Mask', 'Zurich', 'Cologne', 'Incomplete', 'Solve', 'Seven', 'Scandinavian', 'Grandmother', 'Sleep', 'Emotion', 'Salary', 'Nepal', 'Christians', 
    'Sense', 'Nederland', 'Whatsapp', 'Linking', 'Move', 'Santa', 'Sellers', 'Ranking', 'Acceptance', 'Manchester', 'Difference', 'Original','Naming', 'Drink', 'Tips', 'Lithuania', 'Hoping', 'Winning', 'Radiant',
    'Tracks', 'Minecraft', 'Script', 'Installation', 'Bearing', 'Technician', 'Pause', 'Mountain', 'Genetic', 'Stand', 'Older', 'Silicon', 'Appeal', 'Flawless', 'Surprise', 'Polish', 'Embassy', 'Corp', 'Crown', 'Aided', 
    'Monster', 'Meter', 'Reserve', 'Candidate', 'Info', 'Trend', 'Insta', 'Spread', 'Tension', 'Fresh', 'Cambodia', 'Eisenhower', 'Wifi', 'Delay', 'Poster', 'Wait', 'Travelling', 'Contractors', 'Cisco', 'Dirt', 
    'Messaging', 'Reach', 'Intro', 'Projekt', 'Tasting', 'Leeds', 'Annex', 'Dictator', 'Futuristic','Famous', 'Swedish', 'Taxi', 'Obj', 'Signal', 'Mobiles', 'Estimate', 'Pros', 'Extensions', 'Colorado',  
    'Determine', 'Found', 'Confident', 'Merchant', 'Revelation', 'Happily', 'Pack', 'Pool', 'Grateful', 'Volvo', 'Impossible', 'Chapters', 'Trigger', 'Pixel', 'Wonderland', 'Vivid', 'Yesterday', 'Commodity', 'Trading', 
    'Younger', 'Southeast', 'Mention', 'Buddhist', 'Fashionable', 'Luxury', 'Cruise', 'Labour', 'Accept', 'Machinery', 'Covenant', 'Billion', 'Melbourne', 'Biblioteca', 'Driven', 'Caution', 'Hobby', 'Anton', 'Holland', 

#     'Gray', 'West', 'Sales', 'White', 'Green', 'Young', 'Asia', 'Brown', 'Moore',
#     'Student', 'Mars', 'May', 'Asia', 'Low', 'Virginia',
]

stop_wordsY = ['The', 'We', 'This', 'In', 'It', 'Challenge', 'As', 'What', 'Mind', 'For', 'After', 'Design', 'So', 'Our', 'To', 'They', 'My', 'And', 'When', 'By', 
    'Then', 'If', 'With', 'There', 'But', 'One', 'At', 'Since', 'Also', 'Tool', 'While', 'How', 'Once', 'That', 'Each', 'During', 'All', 'From', 'Using', 'Now', 
    'Next', 'On', 'First', 'Innovation', 'Some', 'Business', 'Example', 'He', 'Story', 'You', 'Most', 'India', 'Visual', 'People', 'Company', 'Many', 'Thus', 'Based', 
    'Hence', 'Assignment', 'She', 'Even', 'Map', 'Project', 'An', 'Before', 'Here', 'Every', 'Team', 'Given', 'Is', 'Data', 'Why', 'Sales', 'Due', 'Not', 'Being', 'Use', 
    'Journey', 'Telling', 'English', 'No', 'Week', 'Process', 'New', 'Working', 'Future', 'Well', 'Value', 'More', 'Internet', 'Second', 'Do', 'Moving', 'According', 
    'Service', 'Human', 'Final', 'Good', 'Firstly', 'Post', 'Key', 'Like', 'Other', 'Step', 'Just', 'School', 'Idea', 'Who', 'Time', 'Overall', 'Indian', 'Let', 'Health', 
    'Chain', 'App', 'Finance', 'Both', 'Maps', 'Make', 'Group', 'Later', 'Such', 'Video', 'Your', 'Bank', 'Despite', 'Board', 'Page', 'Its', 'Education', 'Agile', 'Two', 
    'Create', 'Can', 'Going', 'Name', 'Experience', 'Last', 'Peer', 'Brand', 'Ideas', 'Brazil', 'Over', 'Change', 'Quality', 'Tools', 'Only', 'General', 'Student',  #Student
    'Office', 'Director', 'South', 'Gray', 'Technical', 'Her', 'Client', 'Are', 'Market', 'Course', 'Yes', 'High', 'Me', 'Life', 'Model', 'Congress', 'Be', 'Rather', 'Africa', 
    'Or', 'Analysis', 'Knowledge', 'Test', 'Cost', 'Power', 'Along', 'Thank', 'Senior', 'Maybe', 'Growth', 'Head', 'City', 'State', 'Mars', 'Myer', 'United', 'Phase', 'Mexico', 
    'Big', 'Have', 'Getting', 'Start', 'Figure', 'America', 'Apart', 'Works', 'Lack', 'Science', 'Third', 'Kitchen', 'Plan', 'Mobile', 'Define', 'Europe', 'Asia', 'Any', 'Low', 
    'Among', 'Doing', 'Stage', 'Regarding', 'Work', 'Seeing', 'Take', 'Day', 'Again', 'Performance', 'African', 'Center', 'Concept', 'Part', 'Risk', 'Together', 'Children', 
    'Non', 'Does', 'Yet', 'Building', 'Did', 'Meyer', 'Transformation', 'College', 'Co', 'Focus', 'Right', 'Keeping', 'National', 'Smart', 'American', 'Current', 'Drawing', 
    'Care', 'Peru', 'Point', 'Few', 'Out', 'Build', 'Cloud', 'Professor', 'Zoom', 'Teachers', 'Better', 'March', 'Note', 'States', 'End', 'Master', 'Delivery', 
    'Develop', 'Main', 'Below', 'China', 'Web', 'Public', 'Brown', 'Parents', 'Date', 'Members', 'Lead', 'Excel', 'Core', 'Green', 'Liedtka', 'Media', 'Level', 
    'Very', 'Chinese', 'Prior', 'Apple', 'Real', 'Self', 'Platform', 'East', 'Small', 'Impact', 'September', 'Get', 'Elements', 'Spanish', 'Singapore', 'Canvas', 'World', 
    'Learn', 'Wow', 'Store', 'Three', 'Ask', 'Back', 'October', 'Latin', 'Simple', 'Industrial', 'Australia', 'December', 'Add', 'Lean', 'January', 'Central', 'Engineer', 
    'Go', 'Pakistan', 'Intelligence', 'Chief', 'Nigeria', 'Korean', 'Machine', 'Task', 'Keep', 'Increase', 'Number', 'Staff', 'Had', 'Soon', 'President', 'Problems', 'Section', 
    'Coming', 'Easy', 'Latino', 'Amazon', 'Peace', 'Meeting', 'Check', 'Centre', 'North', 'Live', 'Moore', 'Legal', 'Home', 'French', 'Need', 'Persona', 'Mexican', 'November', 
    'See', 'Women', 'York', 'Imagine', 'Flow', 'Paper', 'Pre', 'Best', 'Supply', 'Put', 'Partner', 'Credit', 'Career', 'Price', 'Author', 'Was', 'Fast', 'Job', 'Still',
    'Officer', 'Farmers', 'Content', 'Study', 'Argentina', 'Play', 'Lego', 'Find', 'Open', 'Field', 'Cases', 'Spain', 'Session', 'Share', 'German', 'San', 'Little', 'Young', 
    'Oil', 'Free', 'August', 'Great', 'Tech', 'Leaders', 'Target', 'Loan', 'Consulting', 'Similar', 'Minimum', 'Should', 'Graphic', 'Russia', 'Finding', 'Germany', 'Same', 
    'Coffee', 'Chart', 'Pilot', 'Subject', 'Talent', 'Private', 'Automation', 'Give', 'Selected', 'Various', 'Shop', 'Hiring', 'Write', 'Club', 'Middle', 'Lab', 'Voice', 
    'Around', 'Startup', 'Culture', 'Act', 'Game', 'Plus', 'Scope', 'Friday', 'Up', 'Gas', 'Brands', 'Title', 'Top', 'Brain', 'Area', 'Action', 'Major', 'Junior', 'Wi', 'Fi', 
    'Presentation', 'Air', 'Word', 'Costa', 'Six', 'Draw', 'Travel', 'Banks', 'France', 'Simply', 'Method', 'Physical', 'List', 'Waste', 'Korea', 'Image', 'Energy', 'Campus', 
    'Scientific', 'British', 'Beyond', 'Long', 'Show', 'None', 'Fig', 'Root', 'Net', 'Goal', 'Monday', 'Present', 'Steps', 'Gallery', 'Try', 'Set', 'Creativity', 'Faculty', 
    'Age', 'Select', 'Majority', 'Russian', 'Medium', 'Execution', 'Material', 'Asking', 'Moscow', 'Policy', 'Patients', 'Money', 'Control', 'Tesla', 'Straw', 'Much', 'Allow', 
    'Made', 'Greek', 'Success', 'February', 'Above', 'Sector', 'Christmas', 'Special', 'Benefits', 'Tell', 'Water', 'Line', 'Selling', 'Café', 'Traditional', 'Used', 'Person', 
    'West', 'Japanese', 'Reason', 'Cross', 'Urban', 'Council', 'Less', 'Rapid', 'Beauty', 'Choice', 'Excellence', 'Four', 'Confidential', 'Innovative', 'Visualize', 'Table',
    'Japan', 'Thailand', 'Canada', 'Coaching', 'Choose', 'Reading', 'Agents', 'Local', 'Too', 'Turkey', 'Cause', 'White', 'Payment', 'Order', 'Saturday', 'Survey', 'Role', 
    'Mental', 'Discovery', 'Kenya', 'Plant', 'Portal', 'Law', 'Lunch', 'Built', 'Higher', 'Matter', 'Know', 'Critical', 'Short', 'Videos', 'Accounting', 'Jobs', 'Suddenly', 
    'Family', 'Call', 'Re', 'Cyber', 'Operation', 'Pension', 'Hearing', 'Chile', 'Avoid', 'Ecuador', 'Asset', 'Large', 'Force', 'Mural', 'Standard', 'Positive', 'Child', 'Card', 
    'Setting', 'Honestly', 'Consider', 'Blue', 'Helps', 'Ghana', 'Class', 'Zone', 'Schools', 'Read', 'Multi', 'Room', 'Bachelor', 'Firm', 'Trust', 'Fortune', 'Channel', 'God', 
    'Transport', 'Sale', 'Index', 'Clean', 'Purpose', 'Solar', 'Kids', 'Interview', 'Soldier', 'Columbia', 'Adobe', 'Electric', 'Demand', 'Score', 'Half', 'Needs', 'Western', 
    'Book', 'Events', 'Rica', 'Offer', 'Changes', 'Access', 'Common', 'Primary', 'Eastern', 'Coach', 'Ex', 'Search', 'Mother', 'Speed', 'Corona', 'Doctors', 'Region', 'Grade', 
    'Capital', 'Stress', 'Black', 'Delhi', 'Front', 'Greece', 'Face', 'Tree', 'Radio', 'Estate', 'Serious', 'Topic', 'Discover', 'Laundry', 'Early', 'Active', 'Sample', 'Place', #Hong Kong 
    'Till', 'London', 'Five', 'Branch', 'Alpha', 'Pitch', 'Look', 'Carbon', 'Gender', 'Return', 'Fashion', 'Space', 'Words', 'Uber', 'Probably', 'Talk', 'Member', 'Year', 'Divine', 
    'Manual', 'Teacher', 'Uruguay', 'Previous', 'Trade', 'Drive', 'Press', 'Sub', 'Bus', 'Kingdom', 'Switzerland', 'Nations', 'Airport', 'Scale', 'Answer', 'Clear', 'Patient', 
    'Balance', 'Vehicle', 'True', 'Engine', 'Maersk', 'Certain', 'Walk', 'Hotel', 'Challange', 'Everyday', 'Poor', 'Situation', 'Fail', 'Buyer', 'Device', 'Principal', 'Nature',
    'Labor', 'Electronic', 'Union', 'Desing', 'Natural', 'Done', 'Quito', 'Logo', 'Payroll', 'Slowly', 'Cards', 'Win', 'Tuesday', 'Malaysia', 'Park', 'Einstein', 'Basic', 'Pay', 
    'Buy', 'Studio', 'Movie', 'Hi', 'Lecture', 'Were', 'Federal', 'Quite', 'Past', 'Deep', 'Break', 'Metro', 'Color', 'Breaking', 'Anyway', 'Lots', 'Lower', 'Gain', 'Quick', 'Hall', 
    'Sports', 'Math', 'Musk', 'Dev', 'Fuel', 'Magazine', 'Apps', 'Form', 'Masters', 'Town', 'Remote', 'Juniors', 'Calendar', 'Graphics', 'Bengali', 'Modern', 'Economy', 'Patent', 
    'Property', 'Mr', 'Record', 'California', 'Perfect', 'Normal', 'Billing', 'Rate', 'Coca', 'Cola', 'Parks', 'Drilling', 'Location', 'Rule', 'View', 'Points', 'Venture', 'Provider', 
    'Mid', 'Unique', 'Years', 'Contact', 'Than', 'Degree', 'Chemical', 'Secret', 'Happy', 'Anxiety', 'Traffic', 'Ways', 'Packaging', 'Possible', 'Follow', 'Release', 'Might', 
    'Commerce', 'Chicago', 'Features', 'Aristotle', 'Circle', 'Admin', 'Mumbai', 'Bitcoin', 'Country', 'Maintain', 'Music', 'Parts', 'Profit', 'Journal', 'Guide', 'Off', 'Rural', 
    'Stream', 'Latinos', 'Tax', 'Brief', 'Agreement', 'Father', 'Moderate', 'Pink', 'Full', 'Proof', 'Box', 'Roam', 'Kit', 'Learners', 'Northern', 'Baker', 'Learner', 'Mission', 
    'Gen', 'Uganda', 'Etc', 'Leader', 'Peruvian', 'Egyptian', 'Double', 'House', 'Indians', 'Chapter', 'Bad', 'Jira', 'Pressure', 'Iran', 'Windows', 'Drug', 'Org', 'Micro', 'Eg', 
    'Purchase', 'Zero', 'Meet',
    
#     'George',
]
#'Rubric',

design_people = {'M. Liedtka', 'Jeanne M.', 'Donald Norman', 'Williemien Brand', 'April Greiman', 'Paul Ralph', 'Bernard Roth', 'Tom Kelley', 
                 'Carl DiSalvo', 'Jack Ma', 'Ben Matthews', 'Christoph Meinel', 'Marc Benioff', 'Elena Pacenti', 
                 'Henry Ford', 'Angela Myer', 'Bruce Archer', 'Tom Kelly', 'Brian Lawson', 'Christopher Alexander', 
                 'Lawrence Erlbaum', 'Temple Grandin', 'Kate Crowley', 'Andrew Carnegie', 'Ron Miriello', 'Tony Buzan', 
                 'Laura Fletcher', 'Tim Cook', 'Peter Worthy', 'Roger Martin', 'Stephen Draper', 'Sarah Brooks', 
                 'Lee Vinsel', 'John Jones', 'Sam Walton', 'Bill Burnett', 'Natasha Iskander', 'Tim Ogilvie', 
                 'Max Wertheimer', 'Brad Smith', 'Ralph Caplan', 'Mathew Duerden', 'Jessica Eldridge', 'John Gero', 
                 'Dev Patnaik', 'David Gray', 'Lucy Kimbell', 'Robert Rossma', 'Bill Moggridge', 'Frank Chimero', 
                 'Don Norman', 'Hellen Keller', 'David Kelley', 'Angela Meyer', 'Mary Barra', 'Sunni Brown', 'Tim Chi', 
                 'Bruce Nussbaum', 'Scott Doorley', 'Dave Evans', 'Daniel Pink', 'William Gordon', 'Patrick Hadley', 
                 'Marc Stickdorn', 'Janine Reid', 'Matteo Vignoli', 'Rebecca Ackermann', 'Chad Martin', 'Antionette Carroll', 
                 'Reed Hastings', 'Alex Osborn', 'Saul Bass', 'Jan Chipchase', 'Pete Rowe', 'Robert Curedale', 'Estee Laude', 
                 'Sharon Carmichael', 'CJ Meadows', 'Larry Page', 'Henry Dreyfuss', 'Dieter Rams', 'Ellen Lupton', 'Charvi Parikh', 
                 'Skye Doherty', 'MaeLin Levine', 'Daymond John', 'Vijay Kumar', 'Dave Gray', 'Katie Rast', 'Bo Christensen', 
                 'James Macanufo', 'Robert McKim', 'Jeanne Liedtka', 'Scott Robinson', 'Johansson Sköldberg', 'Warren Buffett', 
                 'Richard Buchanan', 'Jan Auernhammer', 'Rachel Brozenske', 'Sheryl Sandberg', 'Morris Asimow', 'Brian Chesky', 
                 'Nate Spees', 'Sharon Boller', 'Regine Gilbert', 'Michael Dell', 'Donald Schön', 'Elon Musk', 'Jim Bagnall', 
                 'Jeff Weiner', 'Jeremy Myerson', 'Bill Gates', 'Paul Roben', 'Horst Rittel', 'Steve Jobs', 'Alastair Fuad-Luke', 
                 'John Arnold', 'Karen Hold', 'Linden Ball', 'Janine Toole', 'Nick Kelly', 'Robert Peters', 'Patrick Link', 
                 'Herbert Simon', 'Albert Einstein', 'Milton Glaser', 'Robin Mathew', 'Alastair Fuad', 'Frederick Brooks', 
                 'Kees Dorst', 'Richard Florida', 'Matteo Mura', 'Larry Leifer', 'Peter Rowe', 'Don Koberg', 'Uijun Park', 
                 'Sabeen Hussain', 'Melvin Webber', 'Hasso Plattner', 'Erin Patterson', 'Tim Brown', 'Alan Fletcher', 'Niels Diffrient', 
                 'Idris Mootee', 'Walt Disney', 'Alexander Osterwalder', 'Michael Lewrick', 'Erik Edgard', 'Edward Bono', 'Frederik Pferdt', 
                 'Mark Zuckerberg', 'Terry Irwin', 'Michael Beverland', 'Thomas Lockwood', 'Jeff Bezos', 'Alex Osterwalder', 'Joe Gebbia', 
                 'Patti Dobrowolski', 'Walter, Hall', 'Sarah Wilner', 'Bryan Lawson', 'Pietro Micheli', 'Jakob Schneider', 'Scott Edwards', 
                 'Alastair Luke', 'Ezio Manzini', 'Daniel Goleman', 'Liz Sanders', 'Kātz Barry', 'Shelley Goldman', 'Richard Branson', 
                 'Rolf Faste', 'Deborach Szebeko', 'Brian Head', 'Jon Kolko', 'Ralf Speth', 'Nigel Cross', 'Paul Graham', 'Stefan Wiltschnig'}
def check_designers(tokens):
    token = tokens[0]
    tokens_text = token.strip()
    next_token = ''
    if len(tokens) >= 2:
        next_token = tokens[1]
    if next_token.strip() != '':
        tokens_text += ' ' + next_token.strip()
    if tokens_text in design_people:
#         print(token, '           ', tokens_text)
        return True
        
    next_next_token = ''
    if len(tokens) >= 3:
        next_next_token = tokens[2]
    if next_next_token.strip() != '':
        tokens_text += ' ' + next_next_token.strip()
    if tokens_text in design_people:
#         print(token, '           ', tokens_text)
        return True
    return False

stop_words = []
stop_words.extend(stop_words1)
stop_words.extend(stop_words9)
stop_words.extend(stop_words9M)
stop_words.extend(stop_wordsU)
stop_words.extend(stop_wordsF)
stop_words.extend(stop_words10)

stop_words.extend(stop_words2)
stop_words.extend(stop_words3)

if NAMES_SELECTION == 2:
    stop_words.extend(stop_wordsN7)
    stop_words.extend(stop_wordsY)

if NAMES_SELECTION != None:
    stop_wordsU = set(stop_wordsU) ##
    stop_words = set(stop_words) ##
    print('len stop_words: ', len(stop_words)) ##
else:
    stop_wordsU = None
    stop_words = None


def check_for_pii(tokens):
    #return True #disable check
    for i, token in enumerate(tokens):

        if ('google.' in token or 'wikipedia.com' in token or 'trello.com' in token or 'miro.com' in token or 'coursera.org' in token or '.gov' in token):
            continue
            
        if (i < len(tokens)-1 
            and (token == 'Angela' and tokens[i+1] == 'Myer' or token == 'David' and tokens[i+1] == 'Gray' or token == 'Dave' and tokens[i+1] == 'Gray' 
                 or token == 'Tim' and tokens[i+1] == 'Brown' or token == 'Jeanne' and tokens[i+1] == 'Liedtka' or token == 'M.' and tokens[i+1] == 'Liedtka' 
                 or token == 'Jeanne' and tokens[i+1] == 'M.' 
                )
            or token in {'Myer', 'Meyer', 'Brown', 'Gray', 'Lietdka'}):    
            continue
            
        if NAMES_SELECTION != None:
            if token in all_names and token[0].isupper() and token not in stop_words:
                return True
        
        if NAMES_SELECTION == 2:
            #N7 UU:
            if token[0].isupper() and token.lower() not in all_lower_set and token.isalpha() and token[1:].islower() and token not in stop_wordsU and token not in stop_words and token not in all_names:
                return True

        if 'http' in token or 'www.' in token        or '@' in token:
            return True
            
        if NAMES_SELECTION == None or NAMES_SELECTION == 2:
            if token.lower() in {'pin', 'id', 'studentid', 'idstudent', 'pinno'} or token in {'NO', 'Roll', 'ROLL', 'Number', 'Interviewer', 'Interviewee', 'Author', 'AUTHOR'}:
                return True

        if NAMES_SELECTION == None:
            if token[0].isupper() and token.lower() not in all_lower_set and token.isalpha() and token[1:].islower():
                return True
            if token in {'Final', 'Assignment', 'FINAL', 'ASSIGNMENT', 'VISUALIZATION', 'CHALLENGE', 'STORYTELLING', 'MINDMAPPING', 'SELECTION', 'DESIGN', 'REFLECTION', 'DESIGN', 'LAUNCH', 'STORY'}:
                return True
            if token in {'Challenge', 'Mapping', 'Reflection'}:
                return True

        if len(token)>=4 and token[-4:].isdigit() or token == 'ID':
            return True
        
        if token == 'UVA':
            return True
        
    return False


start = time.time()
df = df[df['tokens'].apply(check_for_pii)]
print(len(df))
stop = time.time()
print('df filter: ', stop - start)
train_dataset = Dataset.from_pandas(df, preserve_index=False)
dataset = DatasetDict({'train': train_dataset})
print(dataset)


start = time.time()
dataset = dataset.map(tokenize_and_align_original_columns_new, batched=True, remove_columns=columns_to_remove, num_proc=3)
print('dataset after map tokenize_and_align_original_columns_new function:')
print(dataset)
stop = time.time()
print('dataset.map tokenize_and_align_original_columns_new processing time: ', stop - start)



data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# inference_model_name = base_model
start = time.time()
model = AutoModelForTokenClassification.from_pretrained(inference_model_name, num_labels=len(labels), id2label=id2label, label2id=label2id)
model20 = AutoModelForTokenClassification.from_pretrained(inference_model_name2, num_labels=len(labels), id2label=id2label, label2id=label2id)

stop = time.time()
print('models loading time: ', stop - start)



training_args = TrainingArguments(
    output_dir='my_pii_model',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=7,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    metric_for_best_model='f5',
    greater_is_better=True,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer20 = Trainer(
    model=model20,
    args=training_args,
    train_dataset=dataset['train'],
    tokenizer=tokenizer20,
    data_collator=data_collator,
)


def eval_func(model, test_dataset):
    # torch.set_num_threads(4)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, collate_fn=data_collator) #, num_workers=2)
    test_loader_len = len(test_loader)
    test_preds = []
    test_labels = []

    sess_options = onnxruntime.SessionOptions()
    sess_options.intra_op_num_threads = 1
    session = onnxruntime.InferenceSession(model, providers=['CPUExecutionProvider'], sess_options=sess_options)
    output_layer_name = 'logits'
    results = []
    with torch.no_grad():
        for _, batch in tqdm(enumerate(test_loader), total=test_loader_len):
            input_data = {}
            input_data['input_ids'] = batch['input_ids'].numpy()
            input_data['attention_mask'] = batch['attention_mask'].numpy()
            result = session.run([output_layer_name], input_data) #input_data)
            results.append(result)

        results = np.concatenate(results, axis=1)
        results = np.squeeze(results, axis=0)
    return results

def predict_with_thread_pool(modelo, dataset):

    dataset_with_columns_removed = dataset.remove_columns(['document', 'full_text', 'tokens', 'token_type_ids', 'overflow_to_sample_mapping', 'word_ids'])
    dataset_1 = dataset_with_columns_removed.shard(num_shards=4, index=0, contiguous=True)
    dataset_2 = dataset_with_columns_removed.shard(num_shards=4, index=1, contiguous=True)
    dataset_3 = dataset_with_columns_removed.shard(num_shards=4, index=2, contiguous=True)
    dataset_4 = dataset_with_columns_removed.shard(num_shards=4, index=3, contiguous=True)

    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor: #80s with 4 threads vs 100s with one
        future = executor.submit(eval_func, modelo, dataset_1)
        future2 = executor.submit(eval_func, modelo, dataset_2)
        future3 = executor.submit(eval_func, modelo, dataset_3)
        future4 = executor.submit(eval_func, modelo, dataset_4)

        return_value = future.result()
        return_value2 = future2.result()
        return_value3 = future3.result()
        return_value4 = future4.result()

        predictions = np.concatenate((return_value, return_value2, return_value3, return_value4))
        print('predictions.shape: ', predictions.shape)
    return predictions




start = time.time()
print('torch threads: ', torch.get_num_threads())
torch.set_num_threads(4)
print('torch threads: ', torch.get_num_threads())

if USE_ONNX:
    predictions = predict_with_thread_pool(onnx_model, dataset['train'])
else:
    predictions = trainer.predict(dataset['train']).predictions


stop = time.time()
print('inference part 1 - time elapsed: ', stop - start)


################# nowe
# import torch.nn.functional as F
# predictions = torch.from_numpy(predictions)
# predictions = F.log_softmax(predictions, dim=-1)
# predictions = predictions.numpy()
# print(predictions[0][0])
# #predictions[:, :, 0] -= 885
# predictions[:, :, 2] += 2 #B-ID_NUM
# predictions[:, :, 7] += 2 #B-USERNAME
# predictions[:, :, 8] += 2 #I-ID_NUM
# print(predictions[0][0])
#################

start = time.time()
print('processing predictions part 1...')
predicted_token_class = np.argmax(predictions, axis=-1)
predicted_token_score = np.max(predictions, axis=-1)
# print(predicted_token_class)
print(len(predicted_token_class)) #number of rows in ds
print(len(predicted_token_class[0])) #512

AVG_SCORE_THRESHOLD = 9998.5
print('AVG_SCORE_THRESHOLD: ', AVG_SCORE_THRESHOLD)

token_in_document_dict = {}
csv_rows = []
row_id = 0
non_O_examples = []
high_confidence_rows = []
for i, (example, predicted, scores) in enumerate(zip(dataset['train'], predicted_token_class, predicted_token_score)):
    final_labels = {}
    previous_word_idx = example['word_ids'][0] #it is None
    for word_idx, predicted_label in zip(example['word_ids'], predicted):
        if word_idx != previous_word_idx and word_idx != None:
            if predicted_label != 0:
                final_labels[word_idx] = id2label[predicted_label]
        previous_word_idx = word_idx
        
    scores = [score for score, label in zip(scores, predicted) if label != 0]
    avg_score = sum(scores) / (len(scores) + 1e-8)

    example_added = False
    for token_id, final_label in final_labels.items():
        if final_label != 'O':
            document_id = example['document']
            if avg_score > AVG_SCORE_THRESHOLD:
                high_confidence_rows.append([row_id, document_id, token_id, final_label])
            elif not example_added:
                ###
                token = document_to_rows[document_id]['tokens'][token_id]

                if document_id not in token_in_document_dict:
                    token_in_document_dict[document_id] = {}
                if token not in token_in_document_dict[document_id]:
                    token_in_document_dict[document_id][token] = 1
                else:
                    token_in_document_dict[document_id][token] += 1
                    
                if token_in_document_dict[document_id][token] <= 3:              
                    non_O_examples.append(example)
                    example_added = True
                else:##
                    print('token_in_document_dict more than 3 times: ', document_id, token)##

            csv_rows.append([row_id, document_id, token_id, final_label])
            row_id += 1
stop = time.time()
print('processing predictions part 1. time elapsed: ', stop - start)

           
print('len non_O_examples: ', len(non_O_examples))
print('len high_confidence_rows: ', len(high_confidence_rows))

if True:
    dataset_non_O = Dataset.from_pandas(pd.DataFrame(non_O_examples))
    # dataset_non_O = Dataset.from_list(non_O_examples)
    print(dataset_non_O)

    start = time.time()
    if USE_ONNX:
        predictions = predict_with_thread_pool(onnx_model20, dataset_non_O)
    else:
        predictions = trainer20.predict(dataset_non_O).predictions

    stop = time.time()
    print('dataset_non_O predict: ', stop - start)


    ############################ copy
    start = time.time()
    ################# nowe phase 2
#     import torch.nn.functional as F
#     predictions = torch.from_numpy(predictions)
#     predictions = F.log_softmax(predictions, dim=-1)
#     predictions = predictions.numpy()
#     print(predictions[0][0])
# #     predictions[:, :, 0] -= 1
#     predictions[:, :, 2] += 1 #B-ID_NUM
# #     predictions[:, :, 7] += 6 #B-USERNAME
#     predictions[:, :, 8] += 1 #I-ID_NUM
#     predictions[:, :, 5] += 1 #B-ADDRESS
#     predictions[:, :, 11] += 1 #I-ADDRESS
#     print(predictions[0][0])
    #################
    predicted_token_class = np.argmax(predictions, axis=-1)
    # print(predicted_token_class)
    print(len(predicted_token_class)) #number of rows in ds
    print(len(predicted_token_class[0])) #512

    csv_rows = []
    row_id = 0
    non_O_examples = []
    for i, (example, predicted) in enumerate(zip(dataset_non_O, predicted_token_class)):
        final_labels = {}
        previous_word_idx = example['word_ids'][0] #it is None
        for word_idx, predicted_label in zip(example['word_ids'], predicted):
            if word_idx != previous_word_idx and word_idx != None:
                if predicted_label != 0: #added
                    final_labels[word_idx] = id2label[predicted_label]
            previous_word_idx = word_idx

        example_added = False
        for token_id, final_label in final_labels.items():
            if final_label != 'O':
                if not example_added:
                    non_O_examples.append(example)
                    example_added = True
                document_id = example['document']
                csv_rows.append([row_id, document_id, token_id, final_label])
                row_id += 1
    stop = time.time()
    print('len non_O_examples part 2: ', len(non_O_examples))
    print('processing predictions part 2. time elapsed: ', stop - start)
    ############################ copy


    
print('extending csv_rows with high_confidence_rows from part 1')
csv_rows.extend(high_confidence_rows)
#----------------------------------------------------------------------
csv_rows.extend(csv_simple_rule_rows)
#----------------------------------------------------------------------

            

def simple_unique_fix(csv_rows):
    out_set = set()
    out_list = []
    for row in csv_rows:
        t = (row[1], row[2], row[3])
        if t not in out_set:
            out_list.append(t)
            out_set.add(t)
        
    print('after trivial unique fix: ', len(out_set), len(out_list))

    out_dict = {}
    out_list_cleaned = []
    for t in out_list:
        document = t[0]
        token = t[1]
        label = t[2]
        if (document, token) in out_dict:
            print('skipping duplicate label for: ', (document, token), out_dict[(document, token)], '  ignore:  ', label)
        else:
            out_dict[(document, token)] = label
            
    for k,v in out_dict.items():
        out_list_cleaned.append((k[0], k[1], v))
    out_list_cleaned.sort()
        
    print('after unique fix for inconsistent predictions: ', len(out_list_cleaned))
            
    csv_rows_cleaned = []
    for i, t in enumerate(out_list_cleaned):
        row = [i] + list(t)
        csv_rows_cleaned.append(row)
        # print(row)
        
    return csv_rows_cleaned
    
def fix_b_i_order(csv_rows):
    previous_document = -1
    previous_token = -999
    previous_label = 'NOTHING'
    for i in range(len(csv_rows)):
        row = csv_rows[i]
        t = (row[1], row[2], row[3])   
        document = t[0]
        token = t[1]
        label = t[2]
        if document == previous_document:
            if token != previous_token + 1:
                if 'I-' in label:
                    print('fixing starting label from I- to B-: ', document, token, label)
                    label = label.replace('I-', 'B-')
                    csv_rows[i][3] = label
                
            if token == previous_token + 1:
                if label[2:] == previous_label[2:]:
                    if 'B-' in label:
                        print('fixing continuation label from B- to I-: ', document, token, label)
                        label = label.replace('B-', 'I-')
                        csv_rows[i][3] = label
        
        if document != previous_document:
            if 'I-' in label:
                print('fixing first token occurence in document from I- to B-', document, token, label)
                label = label.replace('I-', 'B-')
                csv_rows[i][3] = label
        
        previous_document = document
        previous_token = token
        previous_label = label



def fix_missed_tokens_by_stats(csv_rows):
    print('fix_missed_tokens_by_stats: creating document to tokens dict:')
    document_to_tokens= {}
    for i, row in tqdm(original_df.iterrows()):
        document_to_tokens[row['document']] = row['tokens']    
    
    document_to_tokens_count = {}
    for csv_row in csv_rows:
        document = csv_row[1]
        token_idx = csv_row[2]
        label = csv_row[3]
        tokens = document_to_tokens[document]
        token = tokens[token_idx]
        if document not in document_to_tokens_count:
            document_to_tokens_count[document] = {}
            
        if token not in document_to_tokens_count[document]:
            document_to_tokens_count[document][token] = 0
        else:
            document_to_tokens_count[document][token] += 1
            
    csv_rows_new = []
    already_added = set()
    for csv_row in csv_rows:
        document = csv_row[1]
        token_idx = csv_row[2]
        label = csv_row[3]
        tokens = document_to_tokens[document]
        token = tokens[token_idx]            
        
        all_token_count = tokens.count(token)
        predicted_token_count = document_to_tokens_count[document][token]
        if all_token_count > 2: #>2 - .96 >1: .957; >3 .96 (best); 4 .96 (worst)
            if len(token.strip()) > 1: #1: >1: .96
                if predicted_token_count / all_token_count > 0: #0.49:
                    token_indexes = [i for i, t in enumerate(tokens) if t == token]
                    for token_idx2 in token_indexes:
                        csv_row_new = csv_row.copy()
                        csv_row_new[2] = token_idx2
                        csv_rows_new.append(csv_row_new)
                        already_added.add((csv_row_new[1], csv_row_new[2]))
                    continue

        if (csv_row[1], csv_row[2]) in already_added:
            continue
        csv_rows_new.append(csv_row.copy())
    return csv_rows_new

def has_digit(s):
    result = False
    for c in s:
        if c.isdigit():
            result = True
            break
    return result

def fix_impossible_predictions(csv_rows):
    
    print('fix_impossible_predictions: creating document to tokens dict:')
    document_to_tokens= {}
    for i, row in tqdm(original_df.iterrows()):
        document_to_tokens[row['document']] = row

    csv_rows_new = []
    idx = 0
    for csv_row in csv_rows:
        document = csv_row[1]
        token_idx = csv_row[2]
        label = csv_row[3]
        tokens = document_to_tokens[document]['tokens']
        token = tokens[token_idx]
        
        start_idx = token_idx - 2
        if start_idx < 0:
            start_idx = 0
        stop_idx = token_idx + 2
        short_text = ' '.join(tokens[start_idx:stop_idx]).replace('\n', ' ')
        i = token_idx ##
        
        csv_row_new = csv_row.copy()
        if '@' in token:
            print('fixing impossible tokens @ -> B-EMAIL', token)
            csv_row_new[3] = 'B-EMAIL'
        elif 'http' in token or 'www.' in token:
            print('fixing impossible tokens http/www -> B-URL_PERSONAL', token)
            csv_row_new[3] = 'B-URL_PERSONAL'
            label = 'B-URL_PERSONAL'
        elif 'NAME_STUDENT' in label and has_digit(token):
            if token[-5:].isdigit():
                print('fixing impossible tokens digit in NAME_STUDENT -> B-ID_NUM', token)
                csv_row_new[3] = 'B-ID_NUM'
            else:
                print('fixing impossible tokens digit in NAME_STUDENT -> B-USERNAME', token)
                csv_row_new[3] = 'B-USERNAME'
        elif 'NAME_STUDENT' in label and token in ['Mrs.', 'Mr.', 'Dr.', 'Ph.D.', 'Ms.', 'Prof.', 'Mrs', 'Mr', 'Dr', 'PhD', 'Ms', 'Prof', '\n\n', '&', '/', ':', ',']:
            print('fixing impossible token word', token)
            csv_row_new = None
        elif label == 'B-URL_PERSONAL' and 'http' not in token and 'www.' not in token:
            print('fixing impossible B-URL_PERSONAL -> 0', token)
            csv_row_new = None
        
#         elif NAMES_SELECTION == 2:
#             if i < len(tokens)-1 and token == 'Will' and tokens[i+1] in {'I', 'Smith', 'apply', 'ask', 'be', 'focus', 'help', 'improve', 'increase', 'it', 'listen', 'look', 'make', 'our', 'permit', 'take', 'the', 'there', 'they', 'try', 'use', 'we', 'you', 'he', 'she'}:
#                 print('fixing WILL -> 0: ', token, tokens[i+1])
#                 csv_row_new = None
#             elif token in {'Myer', 'Meyer', 'Lietdka'}:
#                 print('fixing LECTURERS -> 0: ', token)
#                 csv_row_new = None
#             elif check_designers(tokens[i:i+3]):
#                 print('fixing DESIGNERS -> 0: ', tokens[i:i+3])
#                 csv_row_new = None
#             elif i > 0 and check_designers(tokens[i-1:i+2]):
#                 print('fixing DESIGNERS -> 0: ', tokens[i-1:i+2])
#                 csv_row_new = None
#             elif i > 1 and check_designers(tokens[i-2:i+1]):
#                 print('fixing DESIGNERS -> 0: ', tokens[i-2:i+1])
#                 csv_row_new = None
        
        if 'URL_PERSONAL' in label and ('google.' in token or 'wikipedia.com' in token or 'trello.com' in token or 'miro.com' in token or 'coursera.org' in token or '.gov' in token):
            print('fixing impossible tokens FP -> 0 URL_PERSONAL false positive', token)
            csv_row_new = None
        
        if csv_row_new is not None:
            csv_row_new[0] = idx
            idx += 1
            csv_rows_new.append(csv_row_new)

    return csv_rows_new


csv_rows = fix_missed_tokens_by_stats(csv_rows)

csv_rows = simple_unique_fix(csv_rows)

csv_rows = fix_impossible_predictions(csv_rows)

print('len(csv_rows): ', len(csv_rows))

fix_b_i_order(csv_rows)
print('len(csv_rows): ', len(csv_rows))


csv_header = ['row_id', 'document', 'token', 'label']
out_file = 'submission.csv'

with open(out_file, 'w', encoding='utf-8', newline='\n') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(csv_header)
    csv_writer.writerows(csv_rows)
print(f'output written to {out_file}')
    
total_stop = time.time()
print('total time elapsed: ', total_stop - total_start)
print('done')

starting!


ModuleNotFoundError: No module named 'datasets'